# Chapter 6: Multiple Sequence Alignment Objects

This chapter is about Multiple Sequence Alignments, by which we mean a collection of multiple sequences
which have been aligned together { usually with the insertion of gap characters, and addition of leading or
trailing gaps { such that all the sequence strings are the same length. Such an alignment can be regarded
as a matrix of letters, where each row is held as a SeqRecord object internally.

We will introduce the MultipleSeqAlignment object which holds this kind of data, and the Bio.AlignIO
module for reading and writing them as various le formats (following the design of the Bio.SeqIO module
from the previous chapter). Note that both Bio.SeqIO and Bio.AlignIO can read and write sequence
alignment les. The appropriate choice will depend largely on what you want to do with the data.

The nal part of this chapter is about our command line wrappers for common multiple sequence alignment
tools like ClustalW and MUSCLE.

## 6.1 Parsing or Reading Sequence Alignments

We have two functions for reading in sequence alignments, Bio.AlignIO.read() and Bio.AlignIO.parse()
which following the convention introduced in Bio.SeqIO are for les containing one or multiple alignments
respectively.

Using Bio.AlignIO.parse() will return an iterator which gives MultipleSeqAlignment objects. Iterators
are typically used in a for loop. Examples of situations where you will have multiple dierent
alignments include resampled alignments from the PHYLIP tool seqboot, or multiple pairwise alignments
from the EMBOSS tools water or needle, or Bill Pearson's FASTA tools.

However, in many situations you will be dealing with les which contain only a single alignment. In
this case, you should use the Bio.AlignIO.read() function which returns a single MultipleSeqAlignment
object.

Both functions expect two mandatory arguments:
1. The rst argument is a handle to read the data from, typically an open le (see Section 24.1), or a
lename.
2. The second argument is a lower case string specifying the alignment format. As in Bio.SeqIO we don't
try and guess the le format for you! See http://biopython.org/wiki/AlignIO for a full listing of
supported formats.

There is also an optional seq_count argument which is discussed in Section 6.1.3 below for dealing with
ambiguous le formats which may contain more than one alignment.

### 6.1.1 Single Alignments

As an example, consider the following annotation rich protein alignment in the PFAM or Stockholm le
format:

# STOCKHOLM 1.0
#=GS COATB_BPIKE/30-81 AC P03620.1

#=GS COATB_BPIKE/30-81 DR PDB; 1ifl ; 1-52;

#=GS Q9T0Q8_BPIKE/1-52 AC Q9T0Q8.1

#=GS COATB_BPI22/32-83 AC P15416.1

#=GS COATB_BPM13/24-72 AC P69541.1

#=GS COATB_BPM13/24-72 DR PDB; 2cpb ; 1-49;

#=GS COATB_BPM13/24-72 DR PDB; 2cps ; 1-49;

#=GS COATB_BPZJ2/1-49 AC P03618.1

#=GS Q9T0Q9_BPFD/1-49 AC Q9T0Q9.1

#=GS Q9T0Q9_BPFD/1-49 DR PDB; 1nh4 A; 1-49;

#=GS COATB_BPIF1/22-73 AC P03619.2

#=GS COATB_BPIF1/22-73 DR PDB; 1ifk ; 1-50;

COATB_BPIKE/30-81 AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA

#=GR COATB_BPIKE/30-81 SS -HHHHHHHHHHHHHH--HHHHHHHH--HHHHHHHHHHHHHHHHHHHHH----

Q9T0Q8_BPIKE/1-52 AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA

COATB_BPI22/32-83 DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA

COATB_BPM13/24-72 AEGDDP...AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA

#=GR COATB_BPM13/24-72 SS ---S-T...CHCHHHHCCCCTCCCTTCHHHHHHHHHHHHHHHHHHHHCTT--

COATB_BPZJ2/1-49 AEGDDP...AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA

Q9T0Q9_BPFD/1-49 AEGDDP...AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA

#=GR Q9T0Q9_BPFD/1-49 SS ------...-HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH--

COATB_BPIF1/22-73 FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA

#=GR COATB_BPIF1/22-73 SS XX-HHHH--HHHHHH--HHHHHHH--HHHHHHHHHHHHHHHHHHHHHHH---

#=GC SS_cons XHHHHHHHHHHHHHHHCHHHHHHHHCHHHHHHHHHHHHHHHHHHHHHHHC--

#=GC seq_cons AEssss...AptAhDSLpspAT-hIu.sWshVsslVsAsluIKLFKKFsSKA

//

This is the seed alignment for the Phage Coat Gp8 (PF05371) PFAM entry, downloaded from a now out
of date release of PFAM from https://pfam.xfam.org/. We can load this le as follows (assuming it has
been saved to disk as "PF05371 seed.sth" in the current working directory):

In [1]:
from Bio import AlignIO 
_path = 'data/'
alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")

In [2]:
print(alignment)

Alignment with 7 rows and 52 columns
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRL...SKA COATB_BPIKE/30-81
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKL...SRA Q9T0Q8_BPIKE/1-52
DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRL...SKA COATB_BPI22/32-83
AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA COATB_BPM13/24-72
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA COATB_BPZJ2/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA Q9T0Q9_BPFD/1-49
FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKL...SRA COATB_BPIF1/22-73


You’ll notice in the above output the sequences have been truncated. We could instead write our own code to format this as we please by iterating over the rows as SeqRecord objects:

In [3]:
from Bio import AlignIO
alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")
print("Alignmemt length %i" % alignment.get_alignment_length())

Alignmemt length 52


In [4]:
for record in alignment:
    print("%s - %s" % (record.seq, record.id))

AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA - COATB_BPIKE/30-81
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA - Q9T0Q8_BPIKE/1-52
DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA - COATB_BPI22/32-83
AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA - COATB_BPM13/24-72
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA - COATB_BPZJ2/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA - Q9T0Q9_BPFD/1-49
FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA - COATB_BPIF1/22-73


You could also call Python’s built-in format function on the alignment object to show it in a particular file format – see Section ‍6.2.2 for details.

Did you notice in the raw file above that several of the sequences include database cross-references to the PDB and the associated known secondary structure? Try this:

In [5]:
for record in alignment:
    if record.dbxrefs:
        print("%s %s" % (record.id, record.dbxrefs))

COATB_BPIKE/30-81 ['PDB; 1ifl ; 1-52;']
COATB_BPM13/24-72 ['PDB; 2cpb ; 1-49;', 'PDB; 2cps ; 1-49;']
Q9T0Q9_BPFD/1-49 ['PDB; 1nh4 A; 1-49;']
COATB_BPIF1/22-73 ['PDB; 1ifk ; 1-50;']


To have a look at all the sequence annotation, try this:

In [6]:
for record in alignment:
    print(record)

ID: COATB_BPIKE/30-81
Name: COATB_BPIKE
Description: COATB_BPIKE/30-81
Database cross-references: PDB; 1ifl ; 1-52;
Number of features: 0
/accession=P03620.1
/start=30
/end=81
Per letter annotation for: secondary_structure
Seq('AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA')
ID: Q9T0Q8_BPIKE/1-52
Name: Q9T0Q8_BPIKE
Description: Q9T0Q8_BPIKE/1-52
Number of features: 0
/accession=Q9T0Q8.1
/start=1
/end=52
Seq('AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA')
ID: COATB_BPI22/32-83
Name: COATB_BPI22
Description: COATB_BPI22/32-83
Number of features: 0
/accession=P15416.1
/start=32
/end=83
Seq('DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA')
ID: COATB_BPM13/24-72
Name: COATB_BPM13
Description: COATB_BPM13/24-72
Database cross-references: PDB; 2cpb ; 1-49;, PDB; 2cps ; 1-49;
Number of features: 0
/accession=P69541.1
/start=24
/end=72
Per letter annotation for: secondary_structure
Seq('AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA')
ID: COATB_BPZJ2/1-49
Name: COATB_

PFAM provide a nice web interface at http://pfam.xfam.org/family/PF05371 which will actually let you download this alignment in several other formats. This is what the file looks like in the FASTA file format:

>COATB_BPIKE/30-81
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA
>Q9T0Q8_BPIKE/1-52
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA

>COATB_BPI22/32-83
DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA

>COATB_BPM13/24-72
AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA

>COATB_BPZJ2/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA

>Q9T0Q9_BPFD/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA

>COATB_BPIF1/22-73
FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA

Note the website should have an option about showing gaps as periods (dots) or dashes, we’ve shown dashes above. Assuming you download and save this as file “PF05371_seed.faa” then you can load it with almost exactly the same code:

In [7]:
from Bio import AlignIO
_path = 'data/'
alignment = AlignIO.read(_path + "PF05356_seed.faa", "fasta")

In [8]:
print(alignment)

Alignment with 69 rows and 64 columns
LFMVTDAQAAALVAN......ITVTDIVDQLK..AGAPAIVSIA...GAL I3CK69_9GAMM/17-72
LLFLSSWSVSATPFY......LVVDDVVDQIA..EGAFPVLAIG...GAM I3CE12_9GAMM/11-66
SLLLFSGVASAAPIT......MDVSEVVDQIK..AAAVPILSVG...GAM I3CE11_9GAMM/21-76
VTAALTLPAFAAAAQ......PDVTEVVAYIL..AGIATIALVG...GVL A0A328ZKQ2_9BURK/30-85
LVLASPMALAQTGPT......IDVSSVTGFID.GQLIPGLGTIG...GAA A0A318DYR2_9GAMM/20-76
LLLATSAMASSAFAE......IDVSAATTAIS.TDGSTAITAVG...GAI A0A081KB79_9GAMM/9-65
LLVASALMSSSVFAA......IDISAATTALT.TDGSAAITAVG...GAI A0A081K835_9GAMM/9-65
LAMGTVLSANAAATT......IDTTDVLLSIA..AAVVAIVAVG...AAM A0A1H0GMS3_9BURK/27-82
LSTAAAFVGAQAQAA......IDVTAVGTEIE..AAGNAASSTG...GLI A0A1H4H7Y8_9GAMM/12-67
LSGVALTVGAGLAHA......IDTTKVGASIT..AAETDALTTG...GIV A0A1I1EUT2_9GAMM/16-71
AGFAANAPVFAADGQ......LDTTSVQAGID..AAKATGLSVG...GLV A0A261GG07_9GAMM/20-75
TAVATATLVSATANA......EDLSSITGKINLKSASTGIVAVG...GFL Q7VLI9_HAEDU/13-70
LVTAPALAFAEASTATS....FDVSAITGQISFASVAAGVIAIA...GMI A0A239SNN7_9BURK/21-80
TGMAMSAVAFADDKI...

All that has changed in this code is the filename and the format string. You’ll get the same output as before, the sequences and record identifiers are the same. However, as you should expect, if you check each SeqRecord there is no annotation nor database cross-references because these are not included in the FASTA file format.

Note that rather than using the Sanger website, you could have used Bio.AlignIO to convert the original Stockholm format file into a FASTA file yourself (see below).

With any supported file format, you can load an alignment in exactly the same way just by changing the format string. For example, use “phylip” for PHYLIP files, “nexus” for NEXUS files or “emboss” for the alignments output by the EMBOSS tools. There is a full listing on the wiki page (http://biopython.org/wiki/AlignIO) and in the built in documentation (also online):

In [9]:
from Bio import AlignIO
help(AlignIO)

Help on package Bio.AlignIO in Bio:

NAME
    Bio.AlignIO - Multiple sequence alignment input/output as alignment objects.

DESCRIPTION
    The Bio.AlignIO interface is deliberately very similar to Bio.SeqIO, and in
    fact the two are connected internally.  Both modules use the same set of file
    format names (lower case strings).  From the user's perspective, you can read
    in a PHYLIP file containing one or more alignments using Bio.AlignIO, or you
    can read in the sequences within these alignments using Bio.SeqIO.
    
    Bio.AlignIO is also documented at http://biopython.org/wiki/AlignIO and by
    a whole chapter in our tutorial:
    
    * `HTML Tutorial`_
    * `PDF Tutorial`_
    
    .. _`HTML Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.html
    .. _`PDF Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.pdf
    
    Input
    -----
    For the typical special case when your file or handle contains one and only
    one alignment, use the func

### 6.1.2 Multiple Alignments

The previous section focused on reading files containing a single alignment. In general however, files can contain more than one alignment, and to read these files we must use the Bio.AlignIO.parse() function.

Suppose you have a small alignment in PHYLIP format:

    5      6
            
Alpha     AACAAC

Beta      AACCCC

Gamma     ACCAAC

Delta     CCACCA

Epsilon   CCAAAC

If you wanted to bootstrap a phylogenetic tree using the PHYLIP tools, one of the steps would be to create a set of many resampled alignments using the tool bootseq. This would give output something like this, which has been abbreviated for conciseness:

 5     6
Alpha     AAACCA

Beta      AAACCC

Gamma     ACCCCA

Delta     CCCAAC

Epsilon   CCCAAA

    5     6
Alpha     AAACAA

Beta      AAACCC

Gamma     ACCCAA

Delta     CCCACC

Epsilon   CCCAAA

    5     6
Alpha     AAAAAC

Beta      AAACCC

Gamma     AACAAC

Delta     CCCCCA

Epsilon   CCCAAC

...
    5     6
Alpha     AAAACC

Beta      ACCCCC

Gamma     AAAACC

Delta     CCCCAA

Epsilon   CAAACC

If you wanted to read this in using Bio.AlignIO you could use:

In [20]:
#using Github random.phy
from Bio import AlignIO
_path = 'data/'
alignments = AlignIO.parse(_path + "resampled.phy", "phylip")

In [21]:
#using Github random.phy
for alignment in alignments:
    print(alignment)
    print()

Alignment with 3 rows and 12 columns
ACTGCTAGCTAG Alpha
ACT-CTAGCTAG Beta
ACTGCTAGDTAG Gamma

Alignment with 3 rows and 9 columns
GTCAGC-AG Delta
GACAGCTAG Epsilon
GTCAGCTAG Zeta

Alignment with 3 rows and 13 columns
ACTAGTACAGCTG Eta
ACTAGTACAGCT- Theta
-CTACTACAGGTG Iota



As with the function Bio.SeqIO.parse(), using Bio.AlignIO.parse() returns an iterator. If you want
to keep all the alignments in memory at once, which will allow you to access them in any order, then turn
the iterator into a list:

In [14]:
#doubt
from Bio import AlignIO
_path = 'data/'
alignments = list(AlignIO.parse(_path + "random.phy", "phylip"))
last_align = alignments[-1]
first_align = alignments[0]


<<class 'Bio.Align.MultipleSeqAlignment'> instance (10 records of length 40) at 18f329ae550>

### 6.1.3 Ambiguous Alignments

Many alignment le formats can explicitly store more than one alignment, and the division between each
alignment is clear. However, when a general sequence le format has been used there is no such block
structure. The most common such situation is when alignments have been saved in the FASTA le format.
For example consider the following:

>Alpha
ACTACGACTAGCTCAG--G

>Beta
ACTACCGCTAGCTCAGAAG

>Gamma
ACTACGGCTAGCACAGAAG

>Alpha
ACTACGACTAGCTCAGG--

>Beta
ACTACCGCTAGCTCAGAAG

>Gamma
ACTACGGCTAGCACAGAAG

This could be a single alignment containing six sequences (with repeated identifiers). Or, judging from the identifiers, this is probably two different alignments each with three sequences, which happen to all have the same length.

What about this next example?

>Alpha
ACTACGACTAGCTCAG--G

>Beta
ACTACCGCTAGCTCAGAAG

>Alpha
ACTACGACTAGCTCAGG--

>Gamma
ACTACGGCTAGCACAGAAG

>Alpha
ACTACGACTAGCTCAGG--

>Delta
ACTACGGCTAGCACAGAAG

Again, this could be a single alignment with six sequences. However this time based on the identifiers we might guess this is three pairwise alignments which by chance have all got the same lengths.

This final example is similar:

>Alpha
ACTACGACTAGCTCAG--G

>XXX
ACTACCGCTAGCTCAGAAG

>Alpha
ACTACGACTAGCTCAGG

>YYY
ACTACGGCAAGCACAGG

>Alpha
--ACTACGAC--TAGCTCAGG

>ZZZ
GGACTACGACAATAGCTCAGG

In this third example, because of the differing lengths, this cannot be treated as a single alignment containing all six records. However, it could be three pairwise alignments.

Clearly trying to store more than one alignment in a FASTA file is not ideal. However, if you are forced to deal with these as input files Bio.AlignIO can cope with the most common situation where all the alignments have the same number of records. One example of this is a collection of pairwise alignments, which can be produced by the EMBOSS tools needle and water – although in this situation, Bio.AlignIO should be able to understand their native output using “emboss” as the format string.

To interpret these FASTA examples as several separate alignments, we can use Bio.AlignIO.parse() with the optional seq_count argument which specifies how many sequences are expected in each alignment (in these examples, 3, 2 and 2 respectively). For example, using the third example as the input data:

In [4]:
#doubt

for alignment in AlignIO.parse(handle, "fasta", seq_count=2):
    print("Alignment length %i" % alignment.get_alignment_length())
    for record in alignment:
        print("%s - %s" % (record.seq, record.id))
    print()


NameError: name 'handle' is not defined

Using Bio.AlignIO.read() or Bio.AlignIO.parse() without the seq_count argument would give a single alignment containing all six records for the first two examples. For the third example, an exception would be raised because the lengths differ preventing them being turned into a single alignment.

If the file format itself has a block structure allowing Bio.AlignIO to determine the number of sequences in each alignment directly, then the seq_count argument is not needed. If it is supplied, and doesn’t agree with the file contents, an error is raised.

Note that this optional seq_count argument assumes each alignment in the file has the same number of sequences. Hypothetically you may come across stranger situations, for example a FASTA file containing several alignments each with a different number of sequences – although I would love to hear of a real world example of this. Assuming you cannot get the data in a nicer file format, there is no straight forward way to deal with this using Bio.AlignIO. In this case, you could consider reading in the sequences themselves using Bio.SeqIO and batching them together to create the alignments as appropriate.

## 6.2 Writing Alignments

We’ve talked about using Bio.AlignIO.read() and Bio.AlignIO.parse() for alignment input (reading files), and now we’ll look at Bio.AlignIO.write() which is for alignment output (writing files). This is a function taking three arguments: some MultipleSeqAlignment objects (or for backwards compatibility the obsolete Alignment objects), a handle or filename to write to, and a sequence format.

Here is an example, where we start by creating a few MultipleSeqAlignment objects the hard way (by hand, rather than by loading them from a file). Note we create some SeqRecord objects to construct the alignment from.

In [15]:
 from Bio.Seq import Seq
 from Bio.SeqRecord import SeqRecord
 from Bio.Align import MultipleSeqAlignment
 align1 = MultipleSeqAlignment(
     [
         SeqRecord(Seq("ACTGCTAGCTAG"), id="Alpha"),
         SeqRecord(Seq("ACT-CTAGCTAG"), id="Beta"),
        SeqRecord(Seq("ACTGCTAGDTAG"), id="Gamma"),
    ] 
)

In [16]:
 align2 = MultipleSeqAlignment(
     [
         SeqRecord(Seq("GTCAGC-AG"), id="Delta"),
         SeqRecord(Seq("GACAGCTAG"), id="Epsilon"),
         SeqRecord(Seq("GTCAGCTAG"), id="Zeta"),
     ]
 )

In [17]:
 align3 = MultipleSeqAlignment(
     [
         SeqRecord(Seq("ACTAGTACAGCTG"), id="Eta"),
         SeqRecord(Seq("ACTAGTACAGCT-"), id="Theta"),
         SeqRecord(Seq("-CTACTACAGGTG"), id="Iota"),
     ]
 )

In [18]:
 my_alignments = [align1, align2, align3]

Now we have a list of Alignment objects, we’ll write them to a PHYLIP format file:

In [19]:
# my_example.phy not found so used my_blat.phy
from Bio import AlignIO
_path = 'data/'
AlignIO.write(my_alignments, _path + "my_example.phy", "phylip")

3

Its more common to want to load an existing alignment, and save that, perhaps after some simple manipulation like removing certain rows or columns.

Suppose you wanted to know how many alignments the Bio.AlignIO.write() function wrote to the handle? If your alignments were in a list like the example above, you could just use len(my_alignments), however you can’t do that when your records come from a generator/iterator. Therefore the Bio.AlignIO.write() function returns the number of alignments written to the file.

Note - If you tell the Bio.AlignIO.write() function to write to a file that already exists, the old file will be overwritten without any warning.

### 6.2.1 Converting between sequence alignment file formats

Converting between sequence alignment file formats with Bio.AlignIO works in the same way as converting between sequence file formats with Bio.SeqIO (Section ‍5.5.2). We load generally the alignment(s) using Bio.AlignIO.parse() and then save them using the Bio.AlignIO.write() – or just use the Bio.AlignIO.convert() helper function.

For this example, we’ll load the PFAM/Stockholm format file used earlier and save it as a Clustal W format file:

In [18]:
    from Bio import AlignIO
    _path = 'data/'
    count = AlignIO.convert(_path + "PF05371_seed.sth", "stockholm", "PF05371_seed.aln", "clustal")
    print("Converted %i alignments" % count)
 

Converted 1 alignments


Or, using Bio.AlignIO.parse() and Bio.AlignIO.write():

In [19]:
 from Bio import AlignIO
 _path = 'data/'
 alignments = AlignIO.parse(_path + "PF05371_seed.sth", "stockholm")
 count = AlignIO.write(alignments, "PF05371_seed.aln", "clustal")
 print("Converted %i alignments" % count)

Converted 1 alignments


The Bio.AlignIO.write() function expects to be given multiple alignment objects. In the example above we gave it the alignment iterator returned by Bio.AlignIO.parse().

In this case, we know there is only one alignment in the file so we could have used Bio.AlignIO.read() instead, but notice we have to pass this alignment to Bio.AlignIO.write() as a single element list:

In [20]:
 #doubt
 from Bio import AlignIO
_path = 'data/'
 alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")
 AlignIO.write([alignment], "PF05371_seed.aln", "clustal")

1

Either way, you should end up with the same new Clustal W format file “PF05371_seed.aln” with the following content:

CLUSTAL X (1.81) multiple sequence alignment


COATB_BPIKE/30-81                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSS

Q9T0Q8_BPIKE/1-52                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVS

COATB_BPI22/32-83                   DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSS

COATB_BPM13/24-72                   AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS

COATB_BPZJ2/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFAS

Q9T0Q9_BPFD/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS

COATB_BPIF1/22-73                   FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVS


COATB_BPIKE/30-81                   KA

Q9T0Q8_BPIKE/1-52                   RA

COATB_BPI22/32-83                   KA

COATB_BPM13/24-72                   KA

COATB_BPZJ2/1-49                    KA

Q9T0Q9_BPFD/1-49                    KA

COATB_BPIF1/22-73                   RA

Alternatively, you could make a PHYLIP format file which we’ll name “PF05371_seed.phy”:

In [23]:
#doubt
from Bio import AlignIO
_path = 'data/'
AlignIO.convert(_path + "PF05371_seed.sth", "stockholm", _path + "PF05371_seed.phy", "phylip")

1

One of the big handicaps of the original PHYLIP alignment file format is that the sequence identifiers are strictly truncated at ten characters. In this example, as you can see the resulting names are still unique - but they are not very readable. As a result, a more relaxed variant of the original PHYLIP format is now quite widely used:

In [24]:
 #doubt
 from Bio import AlignIO
 _path = 'data/'
 AlignIO.convert(_path + "PF05371_seed.sth", "stockholm", _path + "PF05371_seed.phy", "phylip-relaxed")

1

If you have to work with the original strict PHYLIP format, then you may need to compress the identifiers somehow – or assign your own names or numbering system. This following bit of code manipulates the record identifiers before saving the output:

In [25]:
 from Bio import AlignIO
 _path = 'data/'
 alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")
 name_mapping = {}
 for i, record in enumerate(alignment):
     name_mapping[i] = record.id
     record.id = "seq%i" % i

 print(name_mapping)

{0: 'COATB_BPIKE/30-81', 1: 'Q9T0Q8_BPIKE/1-52', 2: 'COATB_BPI22/32-83', 3: 'COATB_BPM13/24-72', 4: 'COATB_BPZJ2/1-49', 5: 'Q9T0Q9_BPFD/1-49', 6: 'COATB_BPIF1/22-73'}


In [26]:
AlignIO.write([alignment], "PF05371_seed.phy", "phylip")

1

This code used a Python dictionary to record a simple mapping from the new sequence system to the original identifier:

{
    0: "COATB_BPIKE/30-81",

    1: "Q9T0Q8_BPIKE/1-52",

    2: "COATB_BPI22/32-83",

    # ...
}
Here is the new (strict) PHYLIP format output:

 7 52
seq0       AEPNAATNYA TEAMDSLKTQ AIDLISQTWP VVTTVVVAGL VIRLFKKFSS

seq1       AEPNAATNYA TEAMDSLKTQ AIDLISQTWP VVTTVVVAGL VIKLFKKFVS

seq2       DGTSTATSYA TEAMNSLKTQ ATDLIDQTWP VVTSVAVAGL AIRLFKKFSS

seq3       AEGDDP---A KAAFNSLQAS ATEYIGYAWA MVVVIVGATI GIKLFKKFTS

seq4       AEGDDP---A KAAFDSLQAS ATEYIGYAWA MVVVIVGATI GIKLFKKFAS

seq5       AEGDDP---A KAAFDSLQAS ATEYIGYAWA MVVVIVGATI GIKLFKKFTS

seq6       FAADDATSQA KAAFDSLTAQ ATEMSGYAWA LVVLVVGATV GIKLFKKFVS


           KA

           RA

           KA

           KA

           KA

           KA

           RA
           
In general, because of the identifier limitation, working with strict PHYLIP file formats shouldn’t be your first choice. Using the PFAM/Stockholm format on the other hand allows you to record a lot of additional annotation too.

### 6.2.2 Getting your alignment objects as formatted strings

The Bio.AlignIO interface is based on handles, which means if you want to get your alignment(s) into a string in a particular file format you need to do a little bit more work (see below). However, you will probably prefer to call Python’s built-in format function on the alignment object. This takes an output format specification as a single argument, a lower case string which is supported by Bio.AlignIO as an output format. For example:

In [27]:
 from Bio import AlignIO
 _path = 'data/'
 alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")
 print(format(alignment, "clustal"))

CLUSTAL X (1.81) multiple sequence alignment


COATB_BPIKE/30-81                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSS
Q9T0Q8_BPIKE/1-52                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVS
COATB_BPI22/32-83                   DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSS
COATB_BPM13/24-72                   AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS
COATB_BPZJ2/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFAS
Q9T0Q9_BPFD/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS
COATB_BPIF1/22-73                   FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVS

COATB_BPIKE/30-81                   KA
Q9T0Q8_BPIKE/1-52                   RA
COATB_BPI22/32-83                   KA
COATB_BPM13/24-72                   KA
COATB_BPZJ2/1-49                    KA
Q9T0Q9_BPFD/1-49                    KA
COATB_BPIF1/22-73                   RA





Without an output format specification, format returns the same output as str.

As described in Section ‍4.6, the SeqRecord object has a similar method using output formats supported by Bio.SeqIO.

Internally format is calling Bio.AlignIO.write() with a StringIO handle. You can do this in your own code if for example you are using an older version of Biopython:

In [29]:
 from io import StringIO
 from Bio import AlignIO
 _path = 'data/'
 alignments = AlignIO.parse(_path + "PF05371_seed.sth", "stockholm")
 out_handle = StringIO()
 AlignIO.write(alignments, out_handle, "clustal")

1

In [30]:
 clustal_data = out_handle.getvalue()
 print(clustal_data)

CLUSTAL X (1.81) multiple sequence alignment


COATB_BPIKE/30-81                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSS
Q9T0Q8_BPIKE/1-52                   AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVS
COATB_BPI22/32-83                   DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSS
COATB_BPM13/24-72                   AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS
COATB_BPZJ2/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFAS
Q9T0Q9_BPFD/1-49                    AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTS
COATB_BPIF1/22-73                   FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVS

COATB_BPIKE/30-81                   KA
Q9T0Q8_BPIKE/1-52                   RA
COATB_BPI22/32-83                   KA
COATB_BPM13/24-72                   KA
COATB_BPZJ2/1-49                    KA
Q9T0Q9_BPFD/1-49                    KA
COATB_BPIF1/22-73                   RA





## 6.3 Manipulating Alignments

Now that we’ve covered loading and saving alignments, we’ll look at what else you can do with them.

### 6.3.1 Slicing alignments

First of all, in some senses the alignment objects act like a Python list of SeqRecord objects (the rows). With this model in mind hopefully the actions of len() (the number of rows) and iteration (each row as a SeqRecord) make sense:

In [2]:
 from Bio import AlignIO
 _path = 'data/'
 alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")
 print("Number of rows: %i" % len(alignment))

Number of rows: 7


In [3]:
 for record in alignment:
     print("%s - %s" % (record.seq, record.id))

AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA - COATB_BPIKE/30-81
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA - Q9T0Q8_BPIKE/1-52
DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA - COATB_BPI22/32-83
AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA - COATB_BPM13/24-72
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA - COATB_BPZJ2/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA - Q9T0Q9_BPFD/1-49
FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA - COATB_BPIF1/22-73


You can also use the list-like append and extend methods to add more rows to the alignment (as SeqRecord objects). Keeping the list metaphor in mind, simple slicing of the alignment should also make sense - it selects some of the rows giving back another alignment object:

In [4]:
print(alignment)

Alignment with 7 rows and 52 columns
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRL...SKA COATB_BPIKE/30-81
AEPNAATNYATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKL...SRA Q9T0Q8_BPIKE/1-52
DGTSTATSYATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRL...SKA COATB_BPI22/32-83
AEGDDP---AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA COATB_BPM13/24-72
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA COATB_BPZJ2/1-49
AEGDDP---AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKL...SKA Q9T0Q9_BPFD/1-49
FAADDATSQAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKL...SRA COATB_BPIF1/22-73


What if you wanted to select by column? Those of you who have used the NumPy matrix or array objects won’t be surprised at this - you use a double index.

In [5]:
print(alignment[2, 6])

T


Using two integer indices pulls out a single letter, short hand for this:

In [6]:
print(alignment[2].seq[6])

T


You can pull out a single column as a string like this:

In [7]:
 print(alignment[:, 6])

TTT---T


You can also select a range of columns. For example, to pick out those same three rows we extracted earlier, but take just their first six columns:

In [8]:
print(alignment[3:6, :6])

Alignment with 3 rows and 6 columns
AEGDDP COATB_BPM13/24-72
AEGDDP COATB_BPZJ2/1-49
AEGDDP Q9T0Q9_BPFD/1-49


Leaving the first index as : means take all the rows:

In [9]:
print(alignment[:, :6])

Alignment with 7 rows and 6 columns
AEPNAA COATB_BPIKE/30-81
AEPNAA Q9T0Q8_BPIKE/1-52
DGTSTA COATB_BPI22/32-83
AEGDDP COATB_BPM13/24-72
AEGDDP COATB_BPZJ2/1-49
AEGDDP Q9T0Q9_BPFD/1-49
FAADDA COATB_BPIF1/22-73


This brings us to a neat way to remove a section. Notice columns 7, 8 and 9 which are gaps in three of the seven sequences:

In [10]:
print(alignment[:, 6:9])

Alignment with 7 rows and 3 columns
TNY COATB_BPIKE/30-81
TNY Q9T0Q8_BPIKE/1-52
TSY COATB_BPI22/32-83
--- COATB_BPM13/24-72
--- COATB_BPZJ2/1-49
--- Q9T0Q9_BPFD/1-49
TSQ COATB_BPIF1/22-73


Again, you can slice to get everything after the ninth column:

In [11]:
 print(alignment[:, 9:])

Alignment with 7 rows and 43 columns
ATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA COATB_BPIKE/30-81
ATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA Q9T0Q8_BPIKE/1-52
ATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA COATB_BPI22/32-83
AKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA COATB_BPM13/24-72
AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA COATB_BPZJ2/1-49
AKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA Q9T0Q9_BPFD/1-49
AKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA COATB_BPIF1/22-73


Now, the interesting thing is that addition of alignment objects works by column. This lets you do this as a way to remove a block of columns:

In [12]:
edited = alignment[:, :6] + alignment[:, 9:]
print(edited)

Alignment with 7 rows and 49 columns
AEPNAAATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA COATB_BPIKE/30-81
AEPNAAATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA Q9T0Q8_BPIKE/1-52
DGTSTAATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA COATB_BPI22/32-83
AEGDDPAKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA COATB_BPM13/24-72
AEGDDPAKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA COATB_BPZJ2/1-49
AEGDDPAKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA Q9T0Q9_BPFD/1-49
FAADDAAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA COATB_BPIF1/22-73


Another common use of alignment addition would be to combine alignments for several different genes into a meta-alignment. Watch out though - the identifiers need to match up (see Section ‍4.8 for how adding SeqRecord objects works). You may find it helpful to first sort the alignment rows alphabetically by id:

In [13]:
 edited.sort()
 print(edited)

Alignment with 7 rows and 49 columns
DGTSTAATEAMNSLKTQATDLIDQTWPVVTSVAVAGLAIRLFKKFSSKA COATB_BPI22/32-83
FAADDAAKAAFDSLTAQATEMSGYAWALVVLVVGATVGIKLFKKFVSRA COATB_BPIF1/22-73
AEPNAAATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIRLFKKFSSKA COATB_BPIKE/30-81
AEGDDPAKAAFNSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA COATB_BPM13/24-72
AEGDDPAKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFASKA COATB_BPZJ2/1-49
AEPNAAATEAMDSLKTQAIDLISQTWPVVTTVVVAGLVIKLFKKFVSRA Q9T0Q8_BPIKE/1-52
AEGDDPAKAAFDSLQASATEYIGYAWAMVVVIVGATIGIKLFKKFTSKA Q9T0Q9_BPFD/1-49


Note that you can only add two alignments together if they have the same number of rows.

### 6.3.2 Alignments as arrays

Depending on what you are doing, it can be more useful to turn the alignment object into an array of letters – and you can do this with NumPy:

In [15]:
 import numpy as np
 from Bio import AlignIO
 _path = 'data/'
 alignment = AlignIO.read(_path + "PF05371_seed.sth", "stockholm")
 align_array = np.array([list(rec) for rec in alignment], np.character)
 print("Array shape %i by %i" % align_array.shape)

Array shape 7 by 52


<ipython-input-15-8920656c5e6f>:5: DeprecationWarning: Converting `np.character` to a dtype is deprecated. The current result is `np.dtype(np.str_)` which is not strictly correct. Note that `np.character` is generally deprecated and 'S1' should be used.
  align_array = np.array([list(rec) for rec in alignment], np.character)


If you will be working heavily with the columns, you can tell NumPy to store the array by column (as in Fortran) rather than its default of by row (as in C):

In [16]:
#doubt
align_array = np.array([list(rec) for rec in alignment], np.character, order="F")

<ipython-input-16-2d254d51134c>:1: DeprecationWarning: Converting `np.character` to a dtype is deprecated. The current result is `np.dtype(np.str_)` which is not strictly correct. Note that `np.character` is generally deprecated and 'S1' should be used.
  align_array = np.array([list(rec) for rec in alignment], np.character, order="F")


Note that this leaves the original Biopython alignment object and the NumPy array in memory as separate objects - editing one will not update the other!

## 6.4 Getting information on the alignment

### 6.4.1 Substitutions

The substitutions property of an alignment reports how often letters in the alignment are substituted for each other. This is calculated by taking all pairs of rows in the alignment, counting the number of times two letters are aligned to each other, and summing this over all pairs. For example,

In [17]:
 from Bio.Seq import Seq
 from Bio.SeqRecord import SeqRecord
 from Bio.Align import MultipleSeqAlignment
 alignment = MultipleSeqAlignment(
     [
         SeqRecord(Seq("ACTCCTA"), id='seq1'),
         SeqRecord(Seq("AAT-CTA"), id='seq2'),
         SeqRecord(Seq("CCTACT-"), id='seq3'),
         SeqRecord(Seq("TCTCCTC"), id='seq4'),
     ]
 )

 print(alignment)

Alignment with 4 rows and 7 columns
ACTCCTA seq1
AAT-CTA seq2
CCTACT- seq3
TCTCCTC seq4


In [19]:
 substitutions = alignment.substitutions
 print(substitutions)

    A    C    T
A 2.0  4.5  1.0
C 4.5 10.0  0.5
T 1.0  0.5 12.0



As the ordering of pairs is arbitrary, counts are divided equally above and below the diagonal. For example, the 9 alignments of A to C are stored as 4.5 at position ['A', 'C'] and 4.5 at position ['C', 'A']. This arrangement helps to make the math easier when calculating a substitution matrix from these counts, as described in Section ‍20.4.2.

In [21]:
 m = substitutions.select("ATCG")
 print(m)

    A    T    C   G
A 2.0  1.0  4.5 0.0
T 1.0 12.0  0.5 0.0
C 4.5  0.5 10.0 0.0
G 0.0  0.0  0.0 0.0



This also allows you to change the order of letters in the alphabet.

## 6.5 Alignment Tools

There are lots of algorithms out there for aligning sequences, both pairwise alignments and multiple sequence alignments. These calculations are relatively slow, and you generally wouldn’t want to write such an algorithm in Python. For pairwise alignments Biopython contains the Bio.pairwise2 module , which is supplemented by functions written in C for speed enhancements and the new PairwiseAligner (see Section ‍6.6). In addition, you can use Biopython to invoke a command line tool on your behalf. Normally you would:

Prepare an input file of your unaligned sequences, typically this will be a FASTA file which you might create using Bio.SeqIO (see Chapter ‍5).
Call the command line tool to process this input file, typically via one of Biopython’s command line wrappers (which we’ll discuss here).
Read the output from the tool, i.e. your aligned sequences, typically using Bio.AlignIO (see earlier in this chapter).
All the command line wrappers we’re going to talk about in this chapter follow the same style. You create a command line object specifying the options (e.g. the input filename and the output filename), then invoke this command line via a Python operating system call (e.g. using the subprocess module).

WARNING: We have decided to drop these command line wrappers in a future Biopython release. We will be updating this documentation to instead build the command line directly, and invoke it with the subprocess module.

Most of these wrappers are defined in the Bio.Align.Applications module:

In [23]:
 import Bio.Align.Applications
 dir(Bio.Align.Applications) # doctest:+ELLIPSIS

['ClustalOmegaCommandline',
 'ClustalwCommandline',
 'DialignCommandline',
 'MSAProbsCommandline',
 'MafftCommandline',
 'MuscleCommandline',
 'PrankCommandline',
 'ProbconsCommandline',
 'TCoffeeCommandline',
 '_ClustalOmega',
 '_Clustalw',
 '_Dialign',
 '_MSAProbs',
 '_Mafft',
 '_Muscle',
 '_Prank',
 '_Probcons',
 '_TCoffee',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

(Ignore the entries starting with an underscore – these have special meaning in Python.) The module Bio.Emboss.Applications has wrappers for some of the EMBOSS suite, including needle and water, which are described below in Section ‍6.5.5, and wrappers for the EMBOSS packaged versions of the PHYLIP tools (which EMBOSS refer to as one of their EMBASSY packages - third party tools with an EMBOSS style interface). We won’t explore all these alignment tools here in the section, just a sample, but the same principles apply.

### 6.5.1 ClustalW

ClustalW is a popular command line tool for multiple sequence alignment (there is also a graphical interface called ClustalX). Biopython’s Bio.Align.Applications module has a wrapper for this alignment tool (and several others).

Before trying to use ClustalW from within Python, you should first try running the ClustalW tool yourself by hand at the command line, to familiarise yourself the other options. You’ll find the Biopython wrapper is very faithful to the actual command line API:

In [25]:
 from Bio.Align.Applications import ClustalwCommandline
 help(ClustalwCommandline)

Help on class ClustalwCommandline in module Bio.Align.Applications._Clustalw:

class ClustalwCommandline(Bio.Application.AbstractCommandline)
 |  ClustalwCommandline(cmd='clustalw', **kwargs)
 |  
 |  Command line wrapper for clustalw (version one or two).
 |  
 |  http://www.clustal.org/
 |  
 |  Notes
 |  -----
 |  Last checked against versions: 1.83 and 2.1
 |  
 |  References
 |  ----------
 |  Larkin MA, Blackshields G, Brown NP, Chenna R, McGettigan PA,
 |  McWilliam H, Valentin F, Wallace IM, Wilm A, Lopez R, Thompson JD,
 |  Gibson TJ, Higgins DG. (2007). Clustal W and Clustal X version 2.0.
 |  Bioinformatics, 23, 2947-2948.
 |  
 |  Examples
 |  --------
 |  >>> from Bio.Align.Applications import ClustalwCommandline
 |  >>> in_file = "unaligned.fasta"
 |  >>> clustalw_cline = ClustalwCommandline("clustalw2", infile=in_file)
 |  >>> print(clustalw_cline)
 |  clustalw2 -infile=unaligned.fasta
 |  
 |  You would typically run the command line with clustalw_cline() or via
 |  the

For the most basic usage, all you need is to have a FASTA input file, such as opuntia.fasta (available online or in the Doc/examples subdirectory of the Biopython source code). This is a small FASTA file containing seven prickly-pear DNA sequences (from the cactus family Opuntia).

By default ClustalW will generate an alignment and guide tree file with names based on the input FASTA file, in this case opuntia.aln and opuntia.dnd, but you can override this or make it explicit:

In [28]:
 from Bio.Align.Applications import ClustalwCommandline
 cline = ClustalwCommandline("clustalw2", infile="opuntia.fasta")
print(cline)

clustalw2 -infile=opuntia.fasta


Notice here we have given the executable name as clustalw2, indicating we have version two installed, which has a different filename to version one (clustalw, the default). Fortunately both versions support the same set of arguments at the command line (and indeed, should be functionally identical).

You may find that even though you have ClustalW installed, the above command doesn’t work – you may get a message about “command not found” (especially on Windows). This indicated that the ClustalW executable is not on your PATH (an environment variable, a list of directories to be searched). You can either update your PATH setting to include the location of your copy of ClustalW tools (how you do this will depend on your OS), or simply type in the full path of the tool. For example:

In [29]:
 import os
 from Bio.Align.Applications import ClustalwCommandline
 clustalw_exe = r"C:\Program Files\new clustal\clustalw2.exe"
 clustalw_cline = ClustalwCommandline(clustalw_exe, infile="opuntia.fasta")

In [30]:
 #doubt
 assert os.path.isfile(clustalw_exe), "Clustal W executable missing"
 stdout, stderr = clustalw_cline()

AssertionError: Clustal W executable missing

Remember, in Python strings \n and \t are by default interpreted as a new line and a tab – which is why we’re put a letter “r” at the start for a raw string that isn’t translated in this way. This is generally good practice when specifying a Windows style file name.

Internally this uses the subprocess module which is now the recommended way to run another program in Python. This replaces older options like the os.system() and the os.popen* functions.

Now, at this point it helps to know about how command line tools “work”. When you run a tool at the command line, it will often print text output directly to screen. This text can be captured or redirected, via two “pipes”, called standard output (the normal results) and standard error (for error messages and debug messages). There is also standard input, which is any text fed into the tool. These names get shortened to stdin, stdout and stderr. When the tool finishes, it has a return code (an integer), which by convention is zero for success.

When you run the command line tool like this via the Biopython wrapper, it will wait for it to finish, and check the return code. If this is non zero (indicating an error), an exception is raised. The wrapper then returns two strings, stdout and stderr.

In the case of ClustalW, when run at the command line all the important output is written directly to the output files. Everything normally printed to screen while you wait (via stdout or stderr) is boring and can be ignored (assuming it worked).

What we care about are the two output files, the alignment and the guide tree. We didn’t tell ClustalW what filenames to use, but it defaults to picking names based on the input file. In this case the output should be in the file opuntia.aln. You should be able to work out how to read in the alignment using Bio.AlignIO by now:

In [32]:
 from Bio import AlignIO
 _path = 'data/'
 align = AlignIO.read(_path + "opuntia.aln", "clustal")
 print(align)

Alignment with 7 rows and 906 columns
TATACATTAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273285|gb|AF191659.1|AF191
TATACATTAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273284|gb|AF191658.1|AF191
TATACATTAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273287|gb|AF191661.1|AF191
TATACATAAAAGAAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273286|gb|AF191660.1|AF191
TATACATTAAAGGAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273290|gb|AF191664.1|AF191
TATACATTAAAGGAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273289|gb|AF191663.1|AF191
TATACATTAAAGGAGGGGGATGCGGATAAATGGAAAGGCGAAAG...AGA gi|6273291|gb|AF191665.1|AF191


In case you are interested (and this is an aside from the main thrust of this chapter), the opuntia.dnd file ClustalW creates is just a standard Newick tree file, and Bio.Phylo can parse these:

In [34]:
 from Bio import Phylo
 _path = 'data/'
 tree = Phylo.read(_path + "opuntia.dnd", "newick")
 Phylo.draw_ascii(tree)

                             _______________ gi|6273291|gb|AF191665.1|AF191665
  __________________________|
 |                          |   ______ gi|6273290|gb|AF191664.1|AF191664
 |                          |__|
 |                             |_____ gi|6273289|gb|AF191663.1|AF191663
 |
_|_________________ gi|6273287|gb|AF191661.1|AF191661
 |
 |__________ gi|6273286|gb|AF191660.1|AF191660
 |
 |    __ gi|6273285|gb|AF191659.1|AF191659
 |___|
     | gi|6273284|gb|AF191658.1|AF191658



Chapter 13 covers Biopython’s support for phylogenetic trees in more depth.

### 6.5.2 MUSCLE

MUSCLE is a more recent multiple sequence alignment tool than ClustalW, and Biopython also has a wrapper for it under the Bio.Align.Applications module. As before, we recommend you try using MUSCLE from the command line before trying it from within Python, as the Biopython wrapper is very faithful to the actual command line API:

In [36]:
 from Bio.Align.Applications import MuscleCommandline
 help(MuscleCommandline)

Help on class MuscleCommandline in module Bio.Align.Applications._Muscle:

class MuscleCommandline(Bio.Application.AbstractCommandline)
 |  MuscleCommandline(cmd='muscle', **kwargs)
 |  
 |  Command line wrapper for the multiple alignment program MUSCLE.
 |  
 |  http://www.drive5.com/muscle/
 |  
 |  Notes
 |  -----
 |  Last checked against version: 3.7, briefly against 3.8
 |  
 |  References
 |  ----------
 |  Edgar, Robert C. (2004), MUSCLE: multiple sequence alignment with high
 |  accuracy and high throughput, Nucleic Acids Research 32(5), 1792-97.
 |  
 |  Edgar, R.C. (2004) MUSCLE: a multiple sequence alignment method with
 |  reduced time and space complexity. BMC Bioinformatics 5(1): 113.
 |  
 |  Examples
 |  --------
 |  >>> from Bio.Align.Applications import MuscleCommandline
 |  >>> muscle_exe = r"C:\Program Files\Alignments\muscle3.8.31_i86win32.exe"
 |  >>> in_file = r"C:\My Documents\unaligned.fasta"
 |  >>> out_file = r"C:\My Documents\aligned.fasta"
 |  >>> muscle_cl

For the most basic usage, all you need is to have a FASTA input file, such as opuntia.fasta (available online or in the Doc/examples subdirectory of the Biopython source code). You can then tell MUSCLE to read in this FASTA file, and write the alignment to an output file:

In [37]:
 from Bio.Align.Applications import MuscleCommandline
 _path = 'data/'
 cline = MuscleCommandline(input=_path + "opuntia.fasta", out="opuntia.txt")
 print(cline)

muscle -in data/opuntia.fasta -out opuntia.txt


Note that MUSCLE uses “-in” and “-out” but in Biopython we have to use “input” and “out” as the keyword arguments or property names. This is because “in” is a reserved word in Python.

By default MUSCLE will output the alignment as a FASTA file (using gapped sequences). The Bio.AlignIO module should be able to read this alignment using format="fasta". You can also ask for ClustalW-like output:

In [38]:
 from Bio.Align.Applications import MuscleCommandline
 _path = 'data/'
 cline = MuscleCommandline(input=_path + "opuntia.fasta", out="opuntia.aln", clw=True)
 print(cline)

muscle -in data/opuntia.fasta -out opuntia.aln -clw


The Bio.AlignIO module should be able to read these alignments using format="clustal".

MUSCLE can also output in GCG MSF format (using the msf argument), but Biopython can’t currently parse that, or using HTML which would give a human readable web page (not suitable for parsing).

You can also set the other optional parameters, for example the maximum number of iterations. See the built in help for details.

You would then run MUSCLE command line string as described above for ClustalW, and parse the output using Bio.AlignIO to get an alignment object.

### 6.5.3 MUSCLE using stdout

Using a MUSCLE command line as in the examples above will write the alignment to a file. This means there will be no important information written to the standard out (stdout) or standard error (stderr) handles. However, by default MUSCLE will write the alignment to standard output (stdout). We can take advantage of this to avoid having a temporary output file! For example:

In [39]:
 from Bio.Align.Applications import MuscleCommandline
 _path = 'data/'
 muscle_cline = MuscleCommandline(input=_path + "opuntia.fasta")
 print(muscle_cline)

muscle -in data/opuntia.fasta



If we run this via the wrapper, we get back the output as a string. In order to parse this we can use StringIO to turn it into a handle. Remember that MUSCLE defaults to using FASTA as the output format:

In [44]:
#doubt
from Bio.Align.Applications import MuscleCommandline
_path = 'data/'
muscle_cline = MuscleCommandline(input=_path + "opuntia.fasta")
stdout, stderr = muscle_cline()
from io import StringIO
from Bio import AlignIO
_path = 'data/'
align = AlignIO.read(StringIO(_path + stdout), "fasta")
print(align)

ApplicationError: Non-zero return code 1 from 'muscle -in data/opuntia.fasta', message "'muscle' is not recognized as an internal or external command,"

The above approach is fairly simple, but if you are dealing with very large output text the fact that all of stdout and stderr is loaded into memory as a string can be a potential drawback. Using the subprocess module we can work directly with handles instead:

In [45]:
 import subprocess
 from Bio.Align.Applications import MuscleCommandline
 _path = 'data/'
 muscle_cline = MuscleCommandline(input=_path + "opuntia.fasta")
 child = subprocess.Popen(str(muscle_cline),
                          stdout=subprocess.PIPE,
                          stderr=subprocess.PIPE,
                         universal_newlines=True,
                          shell=(sys.platform!="win32"))
                          

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [46]:
#doubt
 from Bio import AlignIO
 _path = 'data/'
 align = AlignIO.read(_path + child.stdout, "fasta")
 print(align)

NameError: name 'child' is not defined

### 6.5.4 MUSCLE using stdin and stdout

We don’t actually need to have our FASTA input sequences prepared in a file, because by default MUSCLE will read in the input sequence from standard input! Note this is a bit more advanced and fiddly, so don’t bother with this technique unless you need to.

First, we’ll need some unaligned sequences in memory as SeqRecord objects. For this demonstration I’m going to use a filtered version of the original FASTA file (using a generator expression), taking just six of the seven sequences:

In [2]:
 from Bio import SeqIO
 _path = 'data/'
 records = (r for r in SeqIO.parse(_path + "opuntia.fasta", "fasta") if len(r) < 900)

Then we create the MUSCLE command line, leaving the input and output to their defaults (stdin and stdout). I’m also going to ask for strict ClustalW format as for the output.

In [3]:
 from Bio.Align.Applications import MuscleCommandline
 muscle_cline = MuscleCommandline(clwstrict=True)
 print(muscle_cline)

muscle -clwstrict


Now for the fiddly bits using the subprocess module, stdin and stdout:

In [4]:
 import subprocess
 import sys
 child = subprocess.Popen(str(cline),
                          stdin=subprocess.PIPE,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.PIPE,
                          universal_newlines=True,
                          shell=(sys.platform!="win32"))

NameError: name 'cline' is not defined

That should start MUSCLE, but it will be sitting waiting for its FASTA input sequences, which we must supply via its stdin handle:

In [5]:
SeqIO.write(records, child.stdin, "fasta")

NameError: name 'child' is not defined

In [6]:
child.stdin.close()

NameError: name 'child' is not defined

After writing the six sequences to the handle, MUSCLE will still be waiting to see if that is all the FASTA sequences or not – so we must signal that this is all the input data by closing the handle. At that point MUSCLE should start to run, and we can ask for the output:

In [7]:
 from Bio import AlignIO
 align = AlignIO.read(child.stdout, "clustal")
 print(align)

NameError: name 'child' is not defined

Wow! There we are with a new alignment of just the six records, without having created a temporary FASTA input file, or a temporary alignment output file. However, a word of caution: Dealing with errors with this style of calling external programs is much more complicated. It also becomes far harder to diagnose problems, because you can’t try running MUSCLE manually outside of Biopython (because you don’t have the input file to supply). There can also be subtle cross platform issues (e.g. Windows versus Linux), and how you run your script can have an impact (e.g. at the command line, from IDLE or an IDE, or as a GUI script). These are all generic Python issues though, and not specific to Biopython.

If you find working directly with subprocess like this scary, there is an alternative. If you execute the tool with muscle_cline() you can supply any standard input as a big string, muscle_cline(stdin=...). So, provided your data isn’t very big, you can prepare the FASTA input in memory as a string using StringIO (see Section ‍24.1):

In [8]:
 from Bio import SeqIO
 _path = 'data/'
 records = (r for r in SeqIO.parse(_path + "opuntia.fasta", "fasta") if len(r) < 900)
 from io import StringIO
 handle = StringIO()
 SeqIO.write(records, handle, "fasta")

6

In [9]:
data = handle.getvalue()

You can then run the tool and parse the alignment as follows:

In [11]:
 stdout, stderr = muscle_cline(stdin=data)
 from Bio import AlignIO
 align = AlignIO.read(StringIO(stdout), "clustal")
 print(align)

ApplicationError: Non-zero return code 1 from 'muscle -clwstrict', message "'muscle' is not recognized as an internal or external command,"

You might find this easier, but it does require more memory (RAM) for the strings used for the input FASTA and output Clustal formatted data.

### 6.5.5 EMBOSS needle and water

The EMBOSS suite includes the water and needle tools for Smith-Waterman algorithm local alignment, and Needleman-Wunsch global alignment. The tools share the same style interface, so switching between the two is trivial – we’ll just use needle here.

Suppose you want to do a global pairwise alignment between two sequences, prepared in FASTA format as follows:

>HBA_HUMAN

MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHG

KKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTP

AVHASLDKFLASVSTVLTSKYR

in a file alpha.faa, and secondly in a file beta.faa:

>HBB_HUMAN

MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPK

VKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFG

KEFTPPVQAAYQKVVAGVANALAHKYH

You can find copies of these example files with the Biopython source code under the Doc/examples/ directory.

Let’s start by creating a complete needle command line object in one go:

In [12]:
 from Bio.Emboss.Applications import NeedleCommandline
 needle_cline = NeedleCommandline(asequence="alpha.faa", bsequence="beta.faa",
                                  gapopen=10, gapextend=0.5, outfile="needle.txt")
 print(needle_cline)

needle -outfile=needle.txt -asequence=alpha.faa -bsequence=beta.faa -gapopen=10 -gapextend=0.5


Why not try running this by hand at the command prompt? You should see it does a pairwise comparison and records the output in the file needle.txt (in the default EMBOSS alignment file format).

Even if you have EMBOSS installed, running this command may not work – you might get a message about “command not found” (especially on Windows). This probably means that the EMBOSS tools are not on your PATH environment variable. You can either update your PATH setting, or simply tell Biopython the full path to the tool, for example:

In [13]:
 from Bio.Emboss.Applications import NeedleCommandline
 needle_cline = NeedleCommandline(r"C:\EMBOSS\needle.exe",
                                  asequence="alpha.faa", bsequence="beta.faa",
                                  gapopen=10, gapextend=0.5, outfile="needle.txt")

Remember in Python that for a default string \n or \t means a new line or a tab – which is why we’re put a letter “r” at the start for a raw string.

At this point it might help to try running the EMBOSS tools yourself by hand at the command line, to familiarise yourself the other options and compare them to the Biopython help text:

In [14]:
 from Bio.Emboss.Applications import NeedleCommandline
 help(NeedleCommandline)

Help on class NeedleCommandline in module Bio.Emboss.Applications:

class NeedleCommandline(_EmbossCommandLine)
 |  NeedleCommandline(cmd='needle', **kwargs)
 |  
 |  Commandline object for the needle program from EMBOSS.
 |  
 |  Method resolution order:
 |      NeedleCommandline
 |      _EmbossCommandLine
 |      _EmbossMinimalCommandLine
 |      Bio.Application.AbstractCommandline
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cmd='needle', **kwargs)
 |      Initialize the class.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  aformat
 |      Display output in a different specified output format
 |      
 |      This controls the addition of the -aformat parameter and its associated value.  Set this property to the argument value required.
 |  
 |  asequence
 |      First sequence to align
 |      
 |      This controls the addition of the -asequence parameter and its associat

Note that you can also specify (or change or look at) the settings like this:

In [15]:
 from Bio.Emboss.Applications import NeedleCommandline
 needle_cline = NeedleCommandline()
 needle_cline.asequence="alpha.faa"
 needle_cline.bsequence="beta.faa"
 needle_cline.gapopen=10
 needle_cline.gapextend=0.5
 needle_cline.outfile="needle.txt"
 print(needle_cline)

needle -outfile=needle.txt -asequence=alpha.faa -bsequence=beta.faa -gapopen=10 -gapextend=0.5


In [16]:
print(needle_cline.outfile)

needle.txt


Next we want to use Python to run this command for us. As explained above, for full control, we recommend you use the built in Python subprocess module, but for simple usage the wrapper object usually suffices:

In [17]:
 stdout, stderr = needle_cline()
 print(stdout + stderr)

ApplicationError: Non-zero return code 1 from 'needle -outfile=needle.txt -asequence=alpha.faa -bsequence=beta.faa -gapopen=10 -gapextend=0.5', message "'needle' is not recognized as an internal or external command,"

Next we can load the output file with Bio.AlignIO as discussed earlier in this chapter, as the emboss format:

In [19]:
 from Bio import AlignIO
 _path = 'data/'
 align = AlignIO.read(_path + "needle.txt", "emboss")
 print(align)

FileNotFoundError: [Errno 2] No such file or directory: 'data/needle.txt'

In this example, we told EMBOSS to write the output to a file, but you can tell it to write the output to stdout instead (useful if you don’t want a temporary output file to get rid of – use stdout=True rather than the outfile argument), and also to read one of the one of the inputs from stdin (e.g. asequence="stdin", much like in the MUSCLE example in the section above).

This has only scratched the surface of what you can do with needle and water. One useful trick is that the second file can contain multiple sequences (say five), and then EMBOSS will do five pairwise alignments.

## 6.6 Pairwise sequence alignment

Pairwise sequence alignment is the process of aligning two sequences to each other by optimizing the similarity score between them. Biopython includes two built-in pairwise aligners: the ’old’ Bio.pairwise2 module and the new PairwiseAligner class within the Bio.Align module (since Biopython version 1.72). Both can perform global and local alignments and offer numerous options to change the alignment parameters. Although pairwise2 has gained some speed and memory enhancements recently, the new PairwiseAligner is much faster; so if you need to make many alignments with larger sequences, the latter would be the tool to choose. pairwise2, on the contrary, is also able to align lists, which can be useful if your sequences do not consist of single characters only.

Given that the parameters and sequences are the same, both aligners will return the same alignments and alignment score (if the number of alignments is too high they may return different subsets of all valid alignments).


### 6.6.1 pairwise2

Bio.pairwise2 contains essentially the same algorithms as water (local) and needle (global) from the EMBOSS suite (see above) and should return the same results. The pairwise2 module has undergone some optimization regarding speed and memory consumption recently (Biopython versions >1.67) so that for short sequences (global alignments: ~2000 residues, local alignments ~600 residues) it’s faster (or equally fast) to use pairwise2 than calling EMBOSS’ water or needle via the command line tools.

Suppose you want to do a global pairwise alignment between the same two hemoglobin sequences from above (HBA_HUMAN, HBB_HUMAN) stored in alpha.faa and beta.faa:

In [3]:
 #doubt
 from Bio import pairwise2
 from Bio import SeqIO
 _path = 'data/'
 seq1 = SeqIO.read(_path + "alpha.faa", "fasta")
 seq2 = SeqIO.read(_path + "beta.faa", "fasta")
 alignments = pairwise2.align.globalxx(seq1.seq, seq2.seq)

FileNotFoundError: [Errno 2] No such file or directory: 'data/alpha.faa'

As you see, we call the alignment function with align.globalxx. The tricky part are the last two letters of the function name (here: xx), which are used for decoding the scores and penalties for matches (and mismatches) and gaps. The first letter decodes the match score, e.g. x means that a match counts 1 while mismatches have no costs. With m general values for either matches or mismatches can be defined (for more options see Biopython’s API). The second letter decodes the cost for gaps; x means no gap costs at all, with s different penalties for opening and extending a gap can be assigned. So, globalxx means that only matches between both sequences are counted.

Our variable alignments now contains a list of alignments (at least one) which have the same optimal score for the given conditions. In our example this are 80 different alignments with the score 72 (Bio.pairwise2 will return up to 1000 alignments). Have a look at one of these alignments:

In [7]:
from Bio import pairwise2
alignments = pairwise2.align.globalxx("ACCGT", "ACG")

In [8]:
 len(alignments)

2

In [9]:
print(alignments[0])

Alignment(seqA='ACCGT', seqB='A-CG-', score=3.0, start=0, end=5)


Each alignment is a named tuple consisting of the two aligned sequences, the score, the start and the end positions of the alignment (in global alignments the start is always 0 and the end the length of the alignment). Bio.pairwise2 has a function format_alignment for a nicer printout:

In [10]:
print(pairwise2.format_alignment(*alignments[0]))

ACCGT
| || 
A-CG-
  Score=3



Since Biopython 1.77 the required parameters can be supplied with keywords. The last example can now also be written as:

In [11]:
#doubt
alignments = pairwise2.align.globalxx(sequenceA=seq1.seq, sequenceB=seq2.seq)

NameError: name 'seq1' is not defined

Better alignments are usually obtained by penalizing gaps: higher costs for opening a gap and lower costs for extending an existing gap. For amino acid sequences match scores are usually encoded in matrices like PAM or BLOSUM. Thus, a more meaningful alignment for our example can be obtained by using the BLOSUM62 matrix, together with a gap open penalty of 10 and a gap extension penalty of 0.5 (using globalds):

In [14]:
#doubt
 from Bio import pairwise2
 from Bio import SeqIO
 from Bio.Align import substitution_matrices
 blosum62 = substitution_matrices.load("BLOSUM62")
 seq1 = SeqIO.read(_path + "alpha.faa", "fasta")
 seq2 = SeqIO.read("beta.faa", "fasta")
 alignments = pairwise2.align.globalds(seq1.seq, seq2.seq, blosum62, -10, -0.5)
 len(alignments)

FileNotFoundError: [Errno 2] No such file or directory: 'data/alpha.faa'

In [15]:
print(pairwise2.format_alignment(*alignments[0]))

ACCGT
| || 
A-CG-
  Score=3



This alignment has the same score that we obtained earlier with EMBOSS needle using the same sequences and the same parameters.

Local alignments are called similarly with the function align.localXX, where again XX stands for a two letter code for the match and gap functions:

In [16]:
 from Bio import pairwise2
 from Bio.Align import substitution_matrices
 blosum62 = substitution_matrices.load("BLOSUM62")
 alignments = pairwise2.align.localds("LSPADKTNVKAA", "PEEKSAV", blosum62, -10, -1)
 print(pairwise2.format_alignment(*alignments[0]))

3 PADKTNV
  |..|..|
1 PEEKSAV
  Score=16



In recent Biopython versions, format_alignment will only print the aligned part of a local alignment (together with the start positions in 1-based notation, as shown in the above example). If you are also interested in the non- aligned parts of the sequences, use the keyword-parameter full_sequences=True:

In [17]:
 from Bio import pairwise2
 from Bio.Align import substitution_matrices
 blosum62 = substitution_matrices.load("BLOSUM62")
 alignments = pairwise2.align.localds("LSPADKTNVKAA", "PEEKSAV", blosum62, -10, -1)
 print(pairwise2.format_alignment(*alignments[0], full_sequences=True))

LSPADKTNVKAA
  |..|..|   
--PEEKSAV---
  Score=16



Note that local alignments must, as defined by Smith & Waterman, have a positive score (>0). Thus, pairwise2 may return no alignments if no score >0 has been obtained. Also, pairwise2 will not report alignments which are the result of the addition of zero-scoring extensions on either site. In the next example, the pairs serin/aspartate (S/D) and lysin/asparagin (K/N) both have a match score of 0. As you see, the aligned part has not been extended:

In [18]:
 from Bio import pairwise2
 from Bio.Align import substitution_matrices
 blosum62 = substitution_matrices.load("BLOSUM62")
 alignments = pairwise2.align.localds("LSSPADKTNVKKAA", "DDPEEKSAVNN", blosum62, -10, -1)
 print(pairwise2.format_alignment(*alignments[0]))

4 PADKTNV
  |..|..|
3 PEEKSAV
  Score=16



Instead of supplying a complete match/mismatch matrix, the match code m allows for easy defining general match/mismatch values. The next example uses match/mismatch scores of 5/-4 and gap penalties (open/extend) of 2/0.5 using localms:

In [19]:
 alignments = pairwise2.align.localms("AGAACT", "GAC", 5, -4, -2, -0.5)
 print(pairwise2.format_alignment(*alignments[0]))

2 GAAC
  | ||
1 G-AC
  Score=13



One useful keyword argument of the Bio.pairwise2.align functions is score_only. When set to True it will only return the score of the best alignment(s), but in a significantly shorter time. It will also allow the alignment of longer sequences before a memory error is raised. Another useful keyword argument is one_alignment_only=True which will also result in some speed gain.

Unfortunately, Bio.pairwise2 does not work with Biopython’s multiple sequence alignment objects (yet). However, the module has some interesting advanced features: you can define your own match and gap functions (interested in testing affine logarithmic gap costs?), gap penalties and end gaps penalties can be different for both sequences, sequences can be supplied as lists (useful if you have residues that are encoded by more than one character), etc. These features are hard (if at all) to realize with other alignment tools. For more details see the modules documentation in Biopython’s API.

### 6.6.2 PairwiseAligner

The new Bio.Align.PairwiseAligner implements the Needleman-Wunsch, Smith-Waterman, Gotoh (three-state), and Waterman-Smith-Beyer global and local pairwise alignment algorithms. We refer to Durbin et al. [16] for in-depth information on sequence alignment algorithms.

#### 6.6.2.1 Basic usage

To generate pairwise alignments, first create a PairwiseAligner object:

In [20]:
 from Bio import Align
 aligner = Align.PairwiseAligner()

The PairwiseAligner object aligner (see Section ‍6.6.2.2) stores the alignment parameters to be used for the pairwise alignments.

These attributes can be set in the constructor of the object or after the object is made.

In [21]:
aligner = Align.PairwiseAligner(match_score=1.0)

Or, equivalently:

In [22]:
aligner.match_score = 1.0

Use the aligner.score method to calculate the alignment score between two sequences:

In [23]:
 seq1 = "GAACT"
 seq2 = "GAT"
 score = aligner.score(seq1, seq2)
 score

3.0

To see the actual alignments, use the aligner.align method and iterate over the PairwiseAlignment objects returned:

In [25]:
 alignments = aligner.align(seq1, seq2)
 for alignment in alignments:
     print(alignment)

GAACT
||--|
GA--T

GAACT
|-|-|
G-A-T



By default, a global pairwise alignment is performed, which finds the optimal alignment over the whole length of seq1 and seq2. Instead, a local alignment will find the subsequence of seq1 and seq2 with the highest alignment score. Local alignments can be generated by setting aligner.mode to "local":

In [26]:
 aligner.mode = 'local'
 seq1 = "AGAACTC"
 seq2 = "GAACT"
 score = aligner.score(seq1, seq2)
 score

5.0

In [27]:
 alignments = aligner.align(seq1, seq2)
 for alignment in alignments:
     print(alignment)

AGAACTC
 |||||
 GAACT



Note that there is some ambiguity in the definition of the best local alignments if segments with a score 0 can be added to the alignment. We follow the suggestion by Waterman & Eggert [42] and disallow such extensions.

#### 6.6.2.2 The pairwise aligner object

The PairwiseAligner object stores all alignment parameters to be used for the pairwise alignments. To see an overview of the values for all parameters, use

In [28]:
 print(aligner)

Pairwise sequence aligner with parameters
  wildcard: None
  match_score: 1.000000
  mismatch_score: 0.000000
  target_internal_open_gap_score: 0.000000
  target_internal_extend_gap_score: 0.000000
  target_left_open_gap_score: 0.000000
  target_left_extend_gap_score: 0.000000
  target_right_open_gap_score: 0.000000
  target_right_extend_gap_score: 0.000000
  query_internal_open_gap_score: 0.000000
  query_internal_extend_gap_score: 0.000000
  query_left_open_gap_score: 0.000000
  query_left_extend_gap_score: 0.000000
  query_right_open_gap_score: 0.000000
  query_right_extend_gap_score: 0.000000
  mode: local



See Sections ‍6.6.2.3, 6.6.2.4, and 6.6.2.5 below for the definition of these parameters. The attribute mode (described above in Section ‍6.6.2.1) can be set equal to "global" or "local" to specify global or local pairwise alignment, respectively.

Depending on the gap scoring parameters (see Sections ‍6.6.2.4 and 6.6.2.5) and mode, a PairwiseAligner object automatically chooses the appropriate algorithm to use for pairwise sequence alignment. To verify the selected algorithm, use

In [29]:
aligner.algorithm

'Smith-Waterman'

This attribute is read-only.

A PairwiseAligner object also stores the precision є to be used during alignment. The value of є is stored in the attribute aligner.epsilon, and by default is equal to 10−6:

In [30]:
 aligner.epsilon

1e-06

Two scores will be considered equal to each other for the purpose of the alignment if the absolute difference between them is less than є.

#### 6.6.2.3 Substitution scores

Substitution scores define the value to be added to the total score when two letters (nucleotides or amino acids) are aligned to each other. The substitution scores to be used by the PairwiseAligner can be specified in two ways:

By specifying a match score for identical letters, and a mismatch scores for mismatched letters. Nucleotide sequence alignments are typically based on match and mismatch scores. For example, by default BLAST [26] uses a match score of +1 and a mismatch score of −2 for nucleotide alignments by megablast, with a gap penalty of 2.5 (see section 6.6.2.4 for more information on gap scores). Match and mismatch scores can be specified by setting the match and mismatch attributes of the PairwiseAligner object:

In [31]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 aligner.match_score

1.0

In [32]:
aligner.mismatch_score

0.0

In [33]:
 score = aligner.score("ACGT","ACAT")
 print(score)

3.0


In [34]:
 aligner.match_score = 1.0
 aligner.mismatch_score = -2.0
 aligner.gap_score = -2.5
 score = aligner.score("ACGT","ACAT")
 print(score)

1.0


When using match and mismatch scores, you can specify a wildcard character (+None+ by default) for unknown letters. These will get a zero score in alignments, irrespective of the value of the match or mismatch score:

In [35]:
 aligner.wildcard = "?"
 score = aligner.score("ACGT","AC?T")
 print(score)

3.0


Alternatively, you can use the substitution_matrix attribute of the PairwiseAligner object to specify a substitution matrix. This allows you to apply different scores for different pairs of matched and mismatched letters. This is typically used for amino acid sequence alignments. For example, by default BLAST [26] uses the BLOSUM62 substitution matrix for protein alignments by blastp. This substitution matrix is available from Biopython:

In [36]:
 from Bio.Align import substitution_matrices
 substitution_matrices.load()

['BENNER22',
 'BENNER6',
 'BENNER74',
 'BLOSUM45',
 'BLOSUM50',
 'BLOSUM62',
 'BLOSUM80',
 'BLOSUM90',
 'DAYHOFF',
 'FENG',
 'GENETIC',
 'GONNET1992',
 'HOXD70',
 'JOHNSON',
 'JONES',
 'LEVIN',
 'MCLACHLAN',
 'MDM78',
 'NUC.4.4',
 'PAM250',
 'PAM30',
 'PAM70',
 'RAO',
 'RISLER',
 'SCHNEIDER',
 'STR',
 'TRANS']

In [37]:
 matrix = substitution_matrices.load("BLOSUM62")
 print(matrix)

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

In [38]:
 aligner.substitution_matrix = matrix
 score = aligner.score("ACDQ", "ACDQ")
 score

24.0

In [39]:
 score = aligner.score("ACDQ", "ACNQ")
 score

19.0

When using a substitution matrix, X is not interpreted as an unknown character. Instead, the score provided by the substutition matrix will be used:

In [40]:
matrix['D','X']

-1.0

In [41]:
 score = aligner.score("ACDQ", "ACXQ")
 score

17.0

By default, aligner.substitution_matrix is None. The attributes aligner.match_score and aligner.mismatch_score are ignored if aligner.substitution_matrix is not None. Setting aligner.match_score or aligner.mismatch_score to valid values will reset aligner.substitution_matrix to None.

#### 6.6.2.4 Affine gap scores

Affine gap scores are defined by a score to open a gap, and a score to extend an existing gap:

gap score = open gap score + (n−1) × extend gap score,

where n is the length of the gap. Biopython’s pairwise sequence aligner allows fine-grained control over the gap scoring scheme by specifying the following twelve attributes of a PairwiseAligner object:

##### ?table cannot be made
Opening scores	                 Extending scores

query_left_open_gap_score
	     query_left_extend_gap_score

query_internal_open_gap_score
	 query_internal_extend_gap_score

query_right_open_gap_score
	     query_right_extend_gap_score

target_left_open_gap_score
	     target_left_extend_gap_score

target_internal_open_gap_score
	 target_internal_extend_gap_score

target_right_open_gap_score	
     target_right_extend_gap_score
	 


These attributes allow for different gap scores for internal gaps and on either end of the sequence, as shown in this example:


target	query	score
A	-	query left open gap score

C	-	query left extend gap score

C	-	query left extend gap score

G	G	match score

G	T	mismatch score

G	-	query internal open gap score

A	-	query internal extend gap score

A	-	query internal extend gap score

T	T	match score

A	A	match score

G	-	query internal open gap score

C	C	match score

-	C	target internal open gap score

-	C	target internal extend gap score

C	C	match score

T	G	mismatch score

C	C	match score

-	C	target internal open gap score

A	A	match score

-	T	target right open gap score

-	A	target right extend gap score

-	A	target right extend gap score



For convenience, PairwiseAligner objects have additional attributes that refer to a number of these values collectively, as shown (hierarchically) in Table ‍6.1.



Table 6.1: Meta-attributes of the pairwise aligner objects.
Meta-attribute	Attributes it maps to

gap_score	target_gap_score, query_gap_score

open_gap_score	target_open_gap_score, query_open_gap_score

extend_gap_score	target_extend_gap_score, query_extend_gap_score

internal_gap_score	target_internal_gap_score, query_internal_gap_score

internal_open_gap_score	target_internal_open_gap_score, query_internal_open_gap_score

internal_extend_gap_score	target_internal_extend_gap_score, query_internal_extend_gap_score

end_gap_score	target_end_gap_score, query_end_gap_score

end_open_gap_score	target_end_open_gap_score, query_end_open_gap_score

end_extend_gap_score	target_end_extend_gap_score, query_end_extend_gap_score

left_gap_score	target_left_gap_score, query_left_gap_score

right_gap_score	target_right_gap_score, query_right_gap_score

left_open_gap_score	target_left_open_gap_score, query_left_open_gap_score

left_extend_gap_score	target_left_extend_gap_score, query_left_extend_gap_score

right_open_gap_score	target_right_open_gap_score, query_right_open_gap_score

right_extend_gap_score	target_right_extend_gap_score, query_right_extend_gap_score

target_open_gap_score	target_internal_open_gap_score, target_left_open_gap_score,
 	target_right_open_gap_score
target_extend_gap_score	target_internal_extend_gap_score, target_left_extend_gap_score,
 	target_right_extend_gap_score
target_gap_score	target_open_gap_score, target_extend_gap_score

query_open_gap_score	query_internal_open_gap_score, query_left_open_gap_score,
 	query_right_open_gap_score

query_extend_gap_score	query_internal_extend_gap_score, query_left_extend_gap_score,
 	query_right_extend_gap_score

query_gap_score	query_open_gap_score, query_extend_gap_score

target_internal_gap_score	target_internal_open_gap_score, target_internal_extend_gap_score

target_end_gap_score	target_end_open_gap_score, target_end_extend_gap_score

target_end_open_gap_score	target_left_open_gap_score, target_right_open_gap_score

target_end_extend_gap_score	target_left_extend_gap_score, target_right_extend_gap_score

target_left_gap_score	target_left_open_gap_score, target_left_extend_gap_score

target_right_gap_score	target_right_open_gap_score, target_right_extend_gap_score

query_end_gap_score	query_end_open_gap_score, query_end_extend_gap_score

query_end_open_gap_score	query_left_open_gap_score, query_right_open_gap_score

query_end_extend_gap_score	query_left_extend_gap_score, query_right_extend_gap_score

query_internal_gap_score	query_internal_open_gap_score, query_internal_extend_gap_score

query_left_gap_score	query_left_open_gap_score, query_left_extend_gap_score

query_right_gap_score	query_right_open_gap_score, query_right_extend_gap_score



#### 6.6.2.5 General gap scores

For even more fine-grained control over the gap scores, you can specify a gap scoring function. For example, the gap scoring function below disallows a gap after two nucleotides in the query sequence:

In [45]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 def my_gap_score_function(start, length):
     if start==2:
         return -1000
     else:
         return -1 * length

In [46]:
 aligner.query_gap_score = my_gap_score_function
 alignments = aligner.align("AACTT", "AATT")
 for alignment in alignments:
     print(alignment)

AACTT
-|.||
-AATT

AACTT
|-.||
A-ATT

AACTT
||.-|
AAT-T

AACTT
||.|-
AATT-



#### 6.6.2.6 Iterating over alignments

The alignments returned by aligner.align are a kind of immutable iterable objects (similar to range). While they appear similarto a tuple or list of PairwiseAlignment objects, they are different in the sense that each PairwiseAlignment object is created dynamically when it is needed. This approach was chosen because the number of alignments can be extremely large, in particular for poor alignments (see Section ‍6.6.2.9 for an example).

You can perform the following operations on alignments:

len(alignments) returns the number of alignments stored. This function returns quickly, even if the number of alignments is huge. If the number of alignments is extremely large (typically, larger than 9,223,372,036,854,775,807, which is the largest integer that can be stored as a long int on 64 bit machines), len(alignments) will raise an OverflowError. A large number of alignments suggests that the alignment quality is low.

In [47]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 alignments = aligner.align("AAA", "AA")
 len(alignments)

3

You can extract a specific alignment by index:

In [48]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 alignments = aligner.align("AAA", "AA")
 print(alignments[2])

AAA
-||
-AA



You can iterate over alignments, for example as in

In [49]:
 for alignment in alignments:
     print(alignment)

AAA
||-
AA-

AAA
|-|
A-A

AAA
-||
-AA



Note that alignments can be reused, i.e. you can iterate over alignments multiple times:

In [50]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 alignments = aligner.align("AAA", "AA")
 for alignment in alignments:
     print(alignment)

AAA
||-
AA-

AAA
|-|
A-A

AAA
-||
-AA



In [51]:
 for alignment in alignments:
     print(alignment)

AAA
||-
AA-

AAA
|-|
A-A

AAA
-||
-AA



You can also convert the alignments iterator into a list or tuple:

In [52]:
alignments = list(alignments)

It is wise to check the number of alignments by calling len(alignments) before attempting to call list(alignments) to save all alignments as a list.

The alignment score (which has the same value for each alignment in alignments) is stored as an attribute. This allows you to check the alignment score before proceeding to extract individual alignments:

In [58]:
print(alignment.score)

3.0


#### 6.6.2.7 Alignment objects

The aligner.align method returns PairwiseAlignment objects, each representing one alignment between the two sequences.

In [55]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 seq1 = "GAACT"
 seq2 = "GAT"
 alignments = aligner.align(seq1, seq2)
 alignment = alignments[0]
 alignment 

Each alignment stores the alignment score:

In [56]:
 alignment.score

3.0

as well as pointers to the sequences that were aligned:

In [59]:
alignment.target

'GAACT'

In [60]:
alignment.query

'GAT'

Print the PairwiseAlignment object to show the alignment explicitly:

In [61]:
print(alignment)

GAACT
||--|
GA--T



The length of the alignment is defined as the number of aligned sequences, which is always 2 for a pairwise alignment:

In [62]:
len(alignment)

2

The shape property returns a tuple consisting of the length of the alignnment and the number of columns in the alignment as printed:

In [63]:
alignment.shape

(2, 5)

For local alignments, sections that are not aligned are not included in the number of columns:

In [64]:
 aligner.mode = 'local'
 local_alignments = aligner.align("TGAACT", "GAC")
 local_alignment = local_alignments[0]
 print(local_alignment)

TGAACT
 ||-|
 GA-C



Use the aligned property to find the start and end indices of subsequences in the target and query sequence that were aligned to each other. Generally, if the alignment between target (t) and query (q) consists of N chunks, you get two tuples of length N:

In [65]:
alignment.aligned

(((0, 2), (4, 5)), ((0, 2), (2, 3)))

while for the alternative alignment, two tuples of length 3 are returned:

In [66]:
 alignment = alignments[1]
 print(alignment)

GAACT
|-|-|
G-A-T



In [67]:
alignment.aligned

(((0, 1), (2, 3), (4, 5)), ((0, 1), (1, 2), (2, 3)))

Note that different alignments may have the same subsequences aligned to each other. In particular, this may occur if alignments differ from each other in terms of their gap placement only:

In [68]:
 aligner.mode = "global"
 aligner.mismatch_score = -10
 alignments = aligner.align("AAACAAA", "AAAGAAA")
 len(alignments)

2

In [69]:
print(alignments[0])

AAAC-AAA
|||--|||
AAA-GAAA



In [70]:
alignments[0].aligned

(((0, 3), (4, 7)), ((0, 3), (4, 7)))

In [71]:
print(alignments[1])

AAA-CAAA
|||--|||
AAAG-AAA



In [72]:
alignments[1].aligned

(((0, 3), (4, 7)), ((0, 3), (4, 7)))

The aligned property can be used to identify alignments that are identical to each other in terms of their aligned sequences.

The sort method sorts the alignment sequences. By default, sorting is done based on the id attribute of each sequence if available, or the sequence contents otherwise.

In [73]:
print(local_alignment)

TGAACT
 ||-|
 GA-C



In [74]:
 local_alignment.sort()
 print(local_alignment)

 GA-C
 ||-|
TGAACT



Alternatively, you can supply a key function to determine the sort order. For example, you can sort the sequences by increasing GC content:

In [75]:
 from Bio.SeqUtils import GC
 local_alignment.sort(key=GC)
 print(local_alignment)

TGAACT
 ||-|
 GA-C



The reverse argument lets you reverse the sort order to obtain the sequences in decreasing GC content:

In [76]:
 local_alignment.sort(key=GC, reverse=True)
 print(local_alignment)

 GA-C
 ||-|
TGAACT



Use the substitutions method to find the number of substitutions between each pair of nucleotides:

In [4]:
>>> target = "AAAAAAAACCCCCCCCGGGGGGGGTTTTTTTT"
>>> query = "AAAAAAACCCTCCCCGGCCGGGGTTTAGTTT"
>>> alignments = aligner.align(target, query)
>>> aligner.mismatch_score = -1
>>> aligner.gap_score = -1
>>> alignments = aligner.align(target, query)
>>> len(alignments)

NameError: name 'aligner' is not defined

In [78]:
print(alignments[0])

AAAAAAAACCCCCCCCGGGGGGGGTTTTTTTT
|||||||-|||.||||||..|||||||..|||
AAAAAAA-CCCTCCCCGGCCGGGGTTTAGTTT



In [3]:
>>> m = alignments[0].substitutions
>>> print(m)

NameError: name 'alignments' is not defined

Note that the matrix is not symmetric: rows correspond to the target sequence, and columns to the query sequence. For example, the number of G’s in the target sequence that are aligned to a C in the query sequence is

In [80]:
m['G', 'C']

2.0

and the number of C’s in the query sequence tat are aligned to a T in the query sequence is

In [81]:
m['C', 'G']

0.0

To get a symmetric matrix, use

In [82]:
 m += m.transpose()
 m /= 2.0
 print(m)

    A   C   G   T
A 7.0 0.0 0.0 0.5
C 0.0 7.0 1.0 0.5
G 0.0 1.0 6.0 0.5
T 0.5 0.5 0.5 6.0



In [83]:
 m['G', 'C']

1.0

In [84]:
m['C', 'G']

1.0

The total number of substitutions between C’s and G’s in the alignment is 1.0 + 1.0 = 2.

The map method can be applied on a pairwise alignment alignment1 to find the pairwise alignment of the query of alignment2 to the target of alignment1, where the target of alignment2 and the query of alignment1 are identical. A typical example is where alignment1 is the pairwise alignment between a chromosome and a transcript, alignment2 is the pairwise alignment between the transcript and a sequence (e.g., an RNA-seq read), and we want to find the alignment of the sequence to the chromosome:

In [85]:
 aligner.mode = 'local'
 aligner.open_gap_score = -1
 aligner.extend_gap_score = 0
 chromosome = "AAAAAAAACCCCCCCAAAAAAAAAAAGGGGGGAAAAAAAA"
 transcript = "CCCCCCCGGGGGG"
 alignments1 = aligner.align(chromosome, transcript)
 len(alignments1)

1

In [86]:
 alignment1 = alignments1[0]
 print(alignment1)

AAAAAAAACCCCCCCAAAAAAAAAAAGGGGGGAAAAAAAA
        |||||||-----------||||||
        CCCCCCC-----------GGGGGG



In [87]:
 sequence = "CCCCGGGG"
 alignments2 = aligner.align(transcript, sequence)
 len(alignments2)

1

In [88]:
 alignment2 = alignments2[0]
 print(alignment2)

CCCCCCCGGGGGG
   ||||||||
   CCCCGGGG



In [89]:
 mapped_alignment = alignment1.map(alignment2)
 print(mapped_alignment)

AAAAAAAACCCCCCCAAAAAAAAAAAGGGGGGAAAAAAAA
           ||||-----------||||
           CCCC-----------GGGG



In [90]:
format(mapped_alignment, "psl")

'8\t0\t0\t0\t0\t0\t1\t11\t+\tquery\t8\t0\t8\ttarget\t40\t11\t30\t2\t4,4,\t0,4,\t11,26,\n'

Mapping the alignment does not depend on the sequence contents. If we delete the sequence contents, the same alignment is found in PSL format (though we obviously lose the ability to print the sequence alignment):

In [91]:
 from Bio.Seq import Seq
 alignment1.target = Seq(None, len(alignment1.target))
 alignment1.query = Seq(None, len(alignment1.query))
 alignment2.target = Seq(None, len(alignment2.target))
 alignment2.query = Seq(None, len(alignment2.query))
 mapped_alignment = alignment1.map(alignment2)
 format(mapped_alignment, "psl")

'8\t0\t0\t0\t0\t0\t1\t11\t+\tquery\t8\t0\t8\ttarget\t40\t11\t30\t2\t4,4,\t0,4,\t11,26,\n'

##### Slicing and indexing a pairwise alignment

Currently, only slices of the form alignment[:, i:j] are implemented, where i and j are integers or are absent. This returns a new PairwiseAlignment object that includes only the columns i through j in the printed alignment. To illustrate this, in the following example the printed alignment has 5 columns:

In [92]:
print(alignment)

GAACT
|-|-|
G-A-T



Extracting the first 4 columns gives:

In [93]:
alignment[:, :4]

In [94]:
print(alignment[:, :4])

GAACT
|-|-
G-A-T



Here, the final T nucleotides are still shown, but they are not aligned to each other. Note that alignment is a global alignment, but alignment[:, :4] is a local alignment.

Similarly, extracting the last 3 columns gives:

In [95]:
alignment[:, -3:]

In [96]:
 print(alignment[:, -3:])

GAACT
  |-|
 GA-T



This is also now a local alignment, with the initial GA nucleotides in the target and G nucleotide in the query not aligned to each other.

##### Exporting alignments

Use the format method to create a string representation of the alignment in various file formats. This method takes an argument fmt specifying the file format, and may take additional keyword arguments depending on file type. The following values for fmt are supported:

"" (empty string; default): Create a human-readable representation of the alignment (same as when you print the alignment).
"SAM": Create a line representing the alignment in the Sequence Alignment/Map (SAM) format:

>>> alignment.format('sam')
'query\t0\ttarget\t1\t255\t1M1D1M1D1M\t*\t0\t0\tGAT\t*\tAS:i:3\n'
"BED": Create a line representing the alignment in the Browser Extensible Data (BED) file format:

>>> alignment.format('bed')
'target\t0\t5\tquery\t3.0\t+\t0\t5\t0\t3\t1,1,1,\t0,2,4,\n'
"PSL": Create a line representing the alignment in the Pattern Space Layout (PSL) file format as generated by BLAT [29]).

>>> alignment.format('psl')
'3\t0\t0\t0\t0\t0\t2\t2\t+\tquery\t3\t0\t3\ttarget\t5\t0\t5\t3\t1,1,1,\t0,1,2,\t0,2,4,\n'

The first four columns in the PSL output contain the number of matched and mismatched characters, the number of matches to repeat regions, and the number of matches to unknown nucleotides. Repeat regions in the target sequence are indicated by masking the sequence as lower-case or upper-case characters, as defined by the following values for the mask keyword argument:

False (default): Do not count matches to masked sequences separately;

"lower": Count and report matches to lower-case characters as matches to repeat regions;

"upper": Count and report matches to upper-case characters as matches to repeat regions;

The character used for unknown nucleotides is defined by the wildcard argument. For consistency with BLAT, the wildcard character is "N" by default. Use wildcard=None if you don’t want to count matches to any unknown nucleotides separately.

In [97]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 aligner.mismatch_score = -1
 aligner.internal_gap_score = -5
 aligner.wildcard = "N"
 target = "AAAAAAAggggGGNGAAAAA"
 query = "GGTGGGGG"
 alignments = aligner.align(target.upper(), query)
 print(len(alignments))

1


In [98]:
 alignment = alignments[0]
 print(alignment)

AAAAAAAGGGGGGNGAAAAA
-------||.|||.|-----
-------GGTGGGGG-----



In [99]:
alignment.score

5.0

In [100]:
alignment.target

'AAAAAAAGGGGGGNGAAAAA'

In [102]:
 alignment.target = target
 alignment.target

'AAAAAAAggggGGNGAAAAA'

In [103]:
print(alignment)

AAAAAAAggggGGNGAAAAA
-------....||.|-----
-------GGTGGGGG-----



In [104]:
print(alignment.format("psl")) 

6	1	0	1	0	0	0	0	+	query	8	0	8	target	20	7	15	1	8,	0,	7,



In [105]:
print(alignment.format("psl", mask="lower")) 

3	1	3	1	0	0	0	0	+	query	8	0	8	target	20	7	15	1	8,	0,	7,



In [106]:
print(alignment.format("psl", mask="lower", wildcard=None))

3	2	3	0	0	0	0	0	+	query	8	0	8	target	20	7	15	1	8,	0,	7,



In addition to the format method, you can use Python’s built-in format function:

In [107]:
print(format(alignment, "psl"))

6	1	0	1	0	0	0	0	+	query	8	0	8	target	20	7	15	1	8,	0,	7,



allowing PairwiseAlignment objects to be used in formatted (f-) strings in Python:

In [108]:
print(f"The alignment in PSL format is '{alignment:psl}'.")

The alignment in PSL format is '6	1	0	1	0	0	0	0	+	query	8	0	8	target	20	7	15	1	8,	0,	7,
'.


Note that optional keyword arguments cannot be used with the format function or with formatted strings.

#### 6.6.2.8 Aligning to the reverse strand

By default, the pairwise aligner aligns the forward strand of the query to the forward strand of the target. To calculate the alignment score for seq2 to the reverse strand of seq1, use strand="-":

In [109]:
 from Bio import Align
 from Bio.Seq import reverse_complement
 seq1 = "AAAACCC"
 seq2 = "AACC"
 aligner = Align.PairwiseAligner()
 aligner.mismatch_score = -1
 aligner.internal_gap_score = -1
 aligner.score(seq1, seq2) 

4.0

In [110]:
aligner.score(seq1, reverse_complement(seq2), strand="-")

4.0

In [111]:
aligner.score(seq1, seq2, strand="-")

0.0

In [112]:
aligner.score(seq1, reverse_complement(seq2))

0.0

The alignments against the reverse strand can be obtained by specifying strand="-" when calling aligner.align:

In [113]:
 alignments = aligner.align(seq1, seq2)
 len(alignments)

1

In [114]:
print(alignments[0])

AAAACCC
--||||-
--AACC-



In [115]:
print(alignments[0].format("bed"))  

target	2	6	query	4.0	+	2	6	0	1	4,	0,



In [116]:
 alignments = aligner.align(seq1, reverse_complement(seq2), strand="-")
 len(alignments)

1

In [117]:
print(alignments[0])

AAAACCC
--||||-
--AACC-



In [118]:
print(alignments[0].format("bed"))

target	2	6	query	4.0	-	2	6	0	1	4,	0,



In [119]:
 alignments = aligner.align(seq1, seq2, strand="-")
 len(alignments)

2

In [120]:
print(alignments[0])

AAAACCC----
-----------
-------GGTT



In [121]:
print(alignments[1])

----AAAACCC
-----------
GGTT-------



Note that the score for aligning seq2 to the reverse strand of seq1 may be different from the score for aligning the reverse complement of seq2 to the forward strand of seq1 if the left and right gap scores are different:

In [122]:
 aligner.left_gap_score = -0.5
 aligner.right_gap_score = -0.2
 aligner.score(seq1, seq2)

2.8

In [123]:
 alignments = aligner.align(seq1, seq2)
 len(alignments)

1

In [124]:
 print(alignments[0])

AAAACCC
--||||-
--AACC-



In [125]:
aligner.score(seq1, reverse_complement(seq2), strand="-")

3.1

In [126]:
 alignments = aligner.align(seq1, reverse_complement(seq2), strand="-")
 len(alignments)

1

In [127]:
print(alignments[0])

AAAACCC
--||||-
--AACC-



#### 6.6.2.9 Examples

Suppose you want to do a global pairwise alignment between the same two hemoglobin sequences from above (HBA_HUMAN, HBB_HUMAN) stored in alpha.faa and beta.faa:

In [128]:
 #doubt
 from Bio import Align
 from Bio import SeqIO
 seq1 = SeqIO.read("alpha.faa", "fasta")
 seq2 = SeqIO.read("beta.faa", "fasta")
 aligner = Align.PairwiseAligner()
 score = aligner.score(seq1.seq, seq2.seq)
 print(score)

FileNotFoundError: [Errno 2] No such file or directory: 'alpha.faa'

showing an alignment score of 72.0. To see the individual alignments, do

In [129]:
 #doubt
 alignments = aligner.align(seq1.seq, seq2.seq)

AttributeError: 'str' object has no attribute 'seq'

In this example, the total number of optimal alignments is huge (more than 4 × 1037), and calling len(alignments) will raise an OverflowError:

In [132]:
#this code should show error
len(alignments)

1

Let’s have a look at the first alignment:

In [133]:
alignment = alignments[0]

The alignment object stores the alignment score, as well as the alignment itself:

In [134]:
 print(alignment.score)

3.1


In [135]:
print(alignment) 

AAAACCC
--||||-
--AACC-



Better alignments are usually obtained by penalizing gaps: higher costs for opening a gap and lower costs for extending an existing gap. For amino acid sequences match scores are usually encoded in matrices like PAM or BLOSUM. Thus, a more meaningful alignment for our example can be obtained by using the BLOSUM62 matrix, together with a gap open penalty of 10 and a gap extension penalty of 0.5:

In [136]:
 #doubt
 from Bio import Align
 from Bio import SeqIO
 from Bio.Align import substitution_matrices
 seq1 = SeqIO.read("alpha.faa", "fasta")
 seq2 = SeqIO.read("beta.faa", "fasta")
 aligner = Align.PairwiseAligner()
 aligner.open_gap_score = -10
 aligner.extend_gap_score = -0.5
 aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
 score = aligner.score(seq1.seq, seq2.seq)
 print(score)

FileNotFoundError: [Errno 2] No such file or directory: 'alpha.faa'

In [137]:
#doubt
 alignments = aligner.align(seq1.seq, seq2.seq)
 len(alignments)

AttributeError: 'str' object has no attribute 'seq'

In [138]:
print(alignments[0].score)

3.1


In [139]:
print(alignments[0])

AAAACCC
--||||-
--AACC-



This alignment has the same score that we obtained earlier with EMBOSS needle using the same sequences and the same parameters.

To perform a local alignment, set aligner.mode to 'local':

In [140]:
 aligner.mode = 'local'
 aligner.open_gap_score = -10
 aligner.extend_gap_score = -1
 alignments = aligner.align("LSPADKTNVKAA", "PEEKSAV")
 print(len(alignments))

8


In [142]:
alignment = alignments[0]
print(alignment)

   LSPADKTNVKAA
    |
PEEKSAV



In [143]:
 print(alignment.score)

1.0


#### 6.6.2.10 Generalized pairwise alignments

In most cases, PairwiseAligner is used to perform alignments of sequences (strings or Seq objects) consisting of single-letter nucleotides or amino acids. More generally, PairwiseAligner can also be applied to lists or tuples of arbitrary objects. This section will describe some examples of such generalized pairwise alignments.

Generalized pairwise alignments using a substitution matrix and alphabet
Schneider et al. [36] created a substitution matrix for aligning three-nucleotide codons (see below in section 6.7 for more information). This substitution matrix is associated with an alphabet consisting of all three-letter codons:

In [144]:
 from Bio.Align import substitution_matrices
 m = substitution_matrices.load("SCHNEIDER")
 m.alphabet 

('AAA',
 'AAC',
 'AAG',
 'AAT',
 'ACA',
 'ACC',
 'ACG',
 'ACT',
 'AGA',
 'AGC',
 'AGG',
 'AGT',
 'ATA',
 'ATC',
 'ATG',
 'ATT',
 'CAA',
 'CAC',
 'CAG',
 'CAT',
 'CCA',
 'CCC',
 'CCG',
 'CCT',
 'CGA',
 'CGC',
 'CGG',
 'CGT',
 'CTA',
 'CTC',
 'CTG',
 'CTT',
 'GAA',
 'GAC',
 'GAG',
 'GAT',
 'GCA',
 'GCC',
 'GCG',
 'GCT',
 'GGA',
 'GGC',
 'GGG',
 'GGT',
 'GTA',
 'GTC',
 'GTG',
 'GTT',
 'TAA',
 'TAC',
 'TAG',
 'TAT',
 'TCA',
 'TCC',
 'TCG',
 'TCT',
 'TGA',
 'TGC',
 'TGG',
 'TGT',
 'TTA',
 'TTC',
 'TTG',
 'TTT')

We can use this matrix to align codon sequences to each other:

In [146]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
aligner.substitution_matrix = m
 aligner.gap_score = -1.0
 s1 = ('AAT', 'CTG', 'TTT', 'TTT')
 s2 = ('AAT', 'TTA', 'TTT')
 alignments = aligner.align(s1, s2)
 len(alignments)

2

In [147]:
print(alignments[0])

AAT CTG TTT TTT
||| ... ||| ---
AAT TTA TTT ---



In [148]:
 print(alignments[1])

AAT CTG TTT TTT
||| ... --- |||
AAT TTA --- TTT



Note that aligning TTT to TTA, as in this example:

AAT CTG TTT TTT
||| --- ... |||
AAT --- TTA TTT

would get a much lower score:

In [149]:
print(m['CTG', 'TTA'])

7.6


In [150]:
print(m['TTT', 'TTA'])

-0.3


presumably because CTG and TTA both code for leucine, while TTT codes for phenylalanine. The three-letter codon substitution matrix also reveals a preference among codons representing the same amino acid. For example, TTA has a preference for CTG preferred compared to CTC, though all three code for leucine:

In [151]:
 s1 = ('AAT', 'CTG', 'CTC', 'TTT')
 s2 = ('AAT', 'TTA', 'TTT')
 alignments = aligner.align(s1, s2)
 len(alignments)

1

In [152]:
print(alignments[0])

AAT CTG CTC TTT
||| ... --- |||
AAT TTA --- TTT



In [153]:
print(m['CTC', 'TTA'])

6.5


##### Generalized pairwise alignments using match/mismatch scores and an alphabet

Using the three-letter amino acid symbols, the sequences above translate to

In [154]:
 s1 = ('Asn', 'Leu', 'Leu', 'Phe')
 s2 = ('Asn', 'Leu', 'Phe')

We can align these sequences directly to each other by using a three-letter amino acid alphabet:

In [155]:
 from Bio import Align
 aligner = Align.PairwiseAligner()
 aligner.alphabet = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys',
                     'Gln', 'Glu', 'Gly', 'His', 'Ile',
                     'Leu', 'Lys', 'Met', 'Phe', 'Pro',
                     'Ser', 'Thr', 'Trp', 'Tyr', 'Val']

We use +6/-1 match and mismatch scores as an approximation of the BLOSUM62 matrix, and align these sequences to each other:

In [156]:
 aligner.match = +6
 aligner.mismatch = -1
 alignments = aligner.align(s1, s2)
 print(len(alignments))

2


In [157]:
print(alignments[0])

Asn Leu Leu Phe
||| ||| --- |||
Asn Leu --- Phe



In [158]:
 print(alignments[1])

Asn Leu Leu Phe
||| --- ||| |||
Asn --- Leu Phe



In [159]:
print(alignments.score)

18.0


##### Generalized pairwise alignments using match/mismatch scores and integer sequences

Internally, the first step when performing an alignment is to replace the two sequences by integer arrays consisting of the indices of each letter in each sequence in the alphabet associated with the aligner. This step can be bypassed by passing integer arrays directly:

In [160]:
 import numpy
 from Bio import Align
 aligner = Align.PairwiseAligner()
 s1 = numpy.array([2, 10, 10, 13], numpy.int32)
 s2 = numpy.array([2, 10, 13], numpy.int32)
 aligner.match = +6
 aligner.mismatch = -1
 alignments = aligner.align(s1, s2)
 print(len(alignments))

2


In [161]:
print(alignments[0])

2 10 10 13
| || -- ||
2 10 -- 13



In [162]:
print(alignments[1])

2 10 10 13
| -- || ||
2 -- 10 13



In [163]:
 print(alignments.score)

18.0


Note that the indices should consist of 32-bit integers, as specified in this example by numpy.int32.

Unknown letters can again be included by defining a wildcard character, and using the corresponding Unicode code point number as the index:

In [164]:
 aligner.wildcard = "?"
 ord(aligner.wildcard)

63

In [165]:
 s2 = numpy.array([2, 63, 13], numpy.int32)
 aligner.gap_score = -3
 alignments = aligner.align(s1, s2)
 print(len(alignments))

2


In [166]:
print(alignments[0])

2 10 10 13
| .. -- ||
2 63 -- 13



In [167]:
print(alignments[1])

2 10 10 13
| -- .. ||
2 -- 63 13



In [168]:
print(alignments.score)

9.0


##### Generalized pairwise alignments using a substitution matrix and integer sequences

Integer sequences can also be aligned using a substitution matrix, in this case a numpy square array without an alphabet associated with it. In this case, all index values must be non-negative, and smaller than the size of the substitution matrix:

In [169]:
 from Bio import Align
 import numpy
 aligner = Align.PairwiseAligner()
 m = numpy.eye(5)
 m[0, 1:] = m[1:, 0] = -2
 m[2,2] = 3
 print(m)

[[ 1. -2. -2. -2. -2.]
 [-2.  1.  0.  0.  0.]
 [-2.  0.  3.  0.  0.]
 [-2.  0.  0.  1.  0.]
 [-2.  0.  0.  0.  1.]]


In [170]:
 aligner.substitution_matrix = m
 aligner.gap_score = -1
 s1 = numpy.array([0, 2, 3, 4], numpy.int32)
 s2 = numpy.array([0, 3, 2, 1], numpy.int32)
 alignments = aligner.align(s1, s2)
 print(len(alignments))

2


In [171]:
print(alignments[0])

0 - 2 3 4
| - | . -
0 3 2 1 -



In [172]:
print(alignments[1])

0 - 2 3 4
| - | - .
0 3 2 - 1



In [173]:
print(alignments.score)

2.0


## 6.7 Substitution matrices

The Array class in Bio.Align.substitution_matrices is a subclass of numpy arrays that supports indexing both by integers and by specific strings. An Array instance can either be a one-dimensional array or a square two-dimensional arrays. A one-dimensional Array object can for example be used to store the nucleotide frequency of a DNA sequence, while a two-dimensional Array object can be used to represent a scoring matrix for sequence alignments.

### Creating an Array object

To create a one-dimensional Array, only the alphabet of allowed letters needs to be specified:

In [174]:
 from Bio.Align.substitution_matrices import Array
 counts = Array("ACGT")
 print(counts)

A 0.0
C 0.0
G 0.0
T 0.0



The allowed letters are stored in the alphabet property:

In [175]:
counts.alphabet

'ACGT'

This property is read-only; modifying the underlying _alphabet attribute may lead to unexpected results. Elements can be accessed both by letter and by integer index:

In [176]:
 counts['C'] = -3
 counts[2] = 7
 print(counts)

A  0.0
C -3.0
G  7.0
T  0.0



In [177]:
 counts[1]

-3.0

Using a letter that is not in the alphabet, or an index that is out of bounds, will cause a IndexError:

In [180]:
counts['U']

IndexError: 'U'

In [181]:
counts['X'] = 6

IndexError: 'X'

In [182]:
counts[7]

IndexError: index 7 is out of bounds for axis 0 with size 4

A two-dimensional Array can be created by specifying dims=2:

In [183]:
 from Bio.Align.substitution_matrices import Array
 counts = Array("ACGT", dims=2)
 print(counts)

    A   C   G   T
A 0.0 0.0 0.0 0.0
C 0.0 0.0 0.0 0.0
G 0.0 0.0 0.0 0.0
T 0.0 0.0 0.0 0.0



Again, both letters and integers can be used for indexing, and specifying a letter that is not in the alphabet will cause an IndexError:

In [184]:
 counts['A', 'C'] = 12.0
 counts[2, 1] = 5.0
 counts[3, 'T'] = -2
 print(counts)

    A    C   G    T
A 0.0 12.0 0.0  0.0
C 0.0  0.0 0.0  0.0
G 0.0  5.0 0.0  0.0
T 0.0  0.0 0.0 -2.0



In [185]:
counts['X', 1]

IndexError: 'X'

In [186]:
counts['A', 5]

IndexError: index 5 is out of bounds for axis 1 with size 4

Selecting a row or column from the two-dimensional array will return a one-dimensional Array:

In [187]:
 counts = Array("ACGT", dims=2)
 counts['A', 'C'] = 12.0
 counts[2, 1] = 5.0
 counts[3, 'T'] = -2
 counts['G']

Array([0., 5., 0., 0.],
         alphabet='ACGT')

In [188]:
counts[:, 'C']

Array([12.,  0.,  5.,  0.],
         alphabet='ACGT')

Array objects can thus be used as an array and as a dictionary. They can be converted to plain numpy arrays or plain dictionary objects:

In [189]:
 import numpy
 x = Array("ACGT")
 x['C'] = 5
 x

Array([0., 5., 0., 0.],
         alphabet='ACGT')

In [190]:
 a = numpy.array(x)  
 a

array([0., 5., 0., 0.])

In [191]:
 d = dict(x)  # create a plain dictionary
 d

{'A': 0.0, 'C': 5.0, 'G': 0.0, 'T': 0.0}

While the alphabet of an Array is usually a string, you may also use a tuple of (immutable) objects. This is used for example for a codon substitution matrix, where the keys are not individual nucleotides or amino acids but instead three-nucleotide codons.

While the alphabet property of an Array is immutable, you can create a new Array object by selecting the letters you are interested in from the alphabet. For example,

In [192]:
 a = Array("ABCD", dims=2, data=numpy.arange(16).reshape(4,4))
 print(a)

     A    B    C    D
A  0.0  1.0  2.0  3.0
B  4.0  5.0  6.0  7.0
C  8.0  9.0 10.0 11.0
D 12.0 13.0 14.0 15.0



In [193]:
 b = a.select("CAD")
 print(b)

     C    A    D
C 10.0  8.0 11.0
A  2.0  0.0  3.0
D 14.0 12.0 15.0



Note that this also allows you to reorder the alphabet.

Data for letters that are not found in the alphabet are set to zero:

In [194]:
 c = a.select("DEC")
 print(c)

     D   E    C
D 15.0 0.0 14.0
E  0.0 0.0  0.0
C 11.0 0.0 10.0



### Calculating a substitution matrix from a pairwise sequence alignment

As Array is a subclass of a numpy array, you can apply mathematical operations on an Array object in much the same way. Here, we illustrate this by calculating a scoring matrix from the alignment of the 16S ribosomal RNA gene sequences of Escherichia coli and Bacillus subtilis. First, we create a PairwiseAligner and initialize it with the default scores used by blastn:

In [195]:
 from Bio.Align import PairwiseAligner
 aligner = PairwiseAligner()
 aligner.mode = 'local'
 aligner.match_score = 2
 aligner.mismatch_score = -3
 aligner.open_gap_score = -7
 aligner.extend_gap_score = -2

Next, we read in the 16S ribosomal RNA gene sequence of Escherichia coli and Bacillus subtilis (provided in Tests/scoring_matrices/ecoli.fa and Tests/scoring_matrices/bsubtilis.fa), and align them to each other:

In [196]:
 #doubt
 from Bio import SeqIO
 sequence1 = SeqIO.read('ecoli.fa', 'fasta')
 sequence2 = SeqIO.read('bsubtilis.fa', 'fasta')
 alignments = aligner.align(sequence1.seq, sequence2.seq)

FileNotFoundError: [Errno 2] No such file or directory: 'ecoli.fa'

The number of alignments generated is very large:

In [198]:
len(alignments)

2

However, as they only differ trivially from each other, we arbitrarily choose the first alignment, and count the number of each substitution:

In [199]:
 #doubt
 alignment = alignments[0]
 from Bio.Align.substitution_matrices import Array
 frequency = Array("ACGT", dims=2)
 for (start1, end1), (start2, end2) in zip(*alignment.aligned):
     seq1 = sequence1[start1:end1]
     seq2 = sequence2[start2:end2]
     for c1, c2 in zip(seq1, seq2):
         frequency[c1, c2] += 1

 print(frequency)

NameError: name 'sequence1' is not defined

We normalize against the total number to find the probability of each substitution, and create a symmetric matrix:

In [203]:
#check
 import numpy
 probabilities = frequency / numpy.sum(frequency)
 probabilities = (probabilities + probabilities.transpose()) / 2.0
 print(probabilities.format("%.4f"))
    

    A   C   G   T
A nan nan nan nan
C nan nan nan nan
G nan nan nan nan
T nan nan nan nan



The background probability is the probability of finding an A, C, G, or T nucleotide in each sequence separately. This can be calculated as the sum of each row or column:

In [204]:
 #check
 background = numpy.sum(probabilities, 0)
 print(background.format("%.4f"))

A nan
C nan
G nan
T nan



The number of substitutions expected at random is simply the product of the background distribution with itself:

In [205]:
#check
 expected = numpy.dot(background[:,None], background[None, :])
 print(expected.format("%.4f"))

    A   C   G   T
A nan nan nan nan
C nan nan nan nan
G nan nan nan nan
T nan nan nan nan



The scoring matrix can then be calculated as the logarithm of the odds-ratio of the observed and the expected probabilities:

In [206]:
 #check
 oddsratios = probabilities / expected
 scoring_matrix = numpy.log2(oddsratios)
 print(scoring_matrix)

    A   C   G   T
A nan nan nan nan
C nan nan nan nan
G nan nan nan nan
T nan nan nan nan



The matrix can be used to set the substitution matrix for the pairwise aligner:

In [207]:
aligner.substitution_matrix = scoring_matrix

A ValueError is triggered if the Array objects appearing in a mathematical operation have different alphabets:

In [208]:
 from Bio.Align.substitution_matrices import Array
 d = Array("ACGT")
 r = Array("ACGU")
 d + r

ValueError: alphabets are inconsistent

### Reading Array object from file

Bio.Align.substitution_matrices includes a parser to read one- and two-dimensional Array objects from file. One-dimensional arrays are represented by a simple two-column format, with the first column containing the key and the second column the corresponding value. For example, the file hg38.chrom.sizes (obtained from UCSC), available in the Tests/Align subdirectory of the Biopython distribution, contains the size in nucleotides of each chromosome in human genome assembly hg38:

chr1    248956422

chr2    242193529

chr3    198295559

chr4    190214555

...

chrUn_KI270385v1    990

chrUn_KI270423v1    981

chrUn_KI270392v1    971

chrUn_KI270394v1    970

To parse this file, use


In [209]:
 #doubt
 from Bio.Align import substitution_matrices
 with open("hg38.chrom.sizes") as handle:
    table = substitution_matrices.read(handle)

 print(table) 

FileNotFoundError: [Errno 2] No such file or directory: 'hg38.chrom.sizes'

Use dtype=int to read the values as integers:

In [210]:
 #doubt
 with open("hg38.chrom.sizes") as handle:
    table = substitution_matrices.read(handle, int)

 print(table) 

FileNotFoundError: [Errno 2] No such file or directory: 'hg38.chrom.sizes'

For two-dimensional arrays, we follow the file format of substitution matrices provided by NCBI. For example, the BLOSUM62 matrix, which is the default substitution matrix for NCBI’s protein-protein BLAST [26] program blastp, is stored as follows:

##### Matrix made by matblas from blosum62.iij
#####  * column uses minimum score
##### BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#####  Blocks Database = /data/blocks_5.0/blocks.dat
#####  Cluster Percentage: >= 62
#####  Entropy =   0.6979, Expected =  -0.5209
   A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *

A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4 

R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4 

N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4 

D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4 

C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4 

Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4 

E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 

G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4 

H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4 


This file is included in the Biopython distribution under Bio/Align/substitution_matrices/data. To parse this file, use

In [212]:
 #doubt
 from Bio.Align import substitution_matrices
 with open("BLOSUM62") as handle:
   matrix = substitution_matrices.read(handle)

 print(matrix.alphabet)

FileNotFoundError: [Errno 2] No such file or directory: 'BLOSUM62'

In [213]:
print(matrix['A','D'])

-2.0


The header lines starting with # are stored in the attribute header:

In [214]:
matrix.header[0]

'Matrix made by matblas from blosum62.iij'

We can now use this matrix as the substitution matrix on an aligner object:

In [215]:
 from Bio.Align import PairwiseAligner
 aligner = PairwiseAligner()
 aligner.substitution_matrix = matrix

To save an Array object, create a string first:

In [216]:
 text = str(matrix)
 print(text) 

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

and write the text to a file.

### Loading predefined substitution matrices

Biopython contains a large set of substitution matrices defined in the literature, including BLOSUM (Blocks Substitution Matrix) [24] and PAM (Point Accepted Mutation) matrices [14]. These matrices are available as flat files in the Bio/Align/scoring_matrices/data directory, and can be loaded into Python using the load function in the scoring_matrices submodule. For example, the BLOSUM62 matrix can be loaded by running

In [217]:
 from Bio.Align import substitution_matrices
 m = substitution_matrices.load("BLOSUM62")

This substitution matrix has an alphabet consisting of the 20 amino acids used in the genetic code, the three ambiguous amino acids B (asparagine or aspartic acid), Z (glutamine or glutamic acid), and X (representing any amino acid), and the stop codon represented by an asterisk:

In [218]:
m.alphabet

'ARNDCQEGHILKMFPSTWYVBZX*'

To get a full list of available substitution matrices, use load without an argument:



In [219]:
substitution_matrices.load()

['BENNER22',
 'BENNER6',
 'BENNER74',
 'BLOSUM45',
 'BLOSUM50',
 'BLOSUM62',
 'BLOSUM80',
 'BLOSUM90',
 'DAYHOFF',
 'FENG',
 'GENETIC',
 'GONNET1992',
 'HOXD70',
 'JOHNSON',
 'JONES',
 'LEVIN',
 'MCLACHLAN',
 'MDM78',
 'NUC.4.4',
 'PAM250',
 'PAM30',
 'PAM70',
 'RAO',
 'RISLER',
 'SCHNEIDER',
 'STR',
 'TRANS']

Note that the substitution matrix provided by Schneider et al. [36] uses an alphabet consisting of three-nucleotide codons:

In [220]:
 m = substitution_matrices.load("SCHNEIDER")
 m.alphabet

('AAA',
 'AAC',
 'AAG',
 'AAT',
 'ACA',
 'ACC',
 'ACG',
 'ACT',
 'AGA',
 'AGC',
 'AGG',
 'AGT',
 'ATA',
 'ATC',
 'ATG',
 'ATT',
 'CAA',
 'CAC',
 'CAG',
 'CAT',
 'CCA',
 'CCC',
 'CCG',
 'CCT',
 'CGA',
 'CGC',
 'CGG',
 'CGT',
 'CTA',
 'CTC',
 'CTG',
 'CTT',
 'GAA',
 'GAC',
 'GAG',
 'GAT',
 'GCA',
 'GCC',
 'GCG',
 'GCT',
 'GGA',
 'GGC',
 'GGG',
 'GGT',
 'GTA',
 'GTC',
 'GTG',
 'GTT',
 'TAA',
 'TAC',
 'TAG',
 'TAT',
 'TCA',
 'TCC',
 'TCG',
 'TCT',
 'TGA',
 'TGC',
 'TGG',
 'TGT',
 'TTA',
 'TTC',
 'TTG',
 'TTT')